# Question Formulation 
You should refine the question formulation based on the general task description you’ve been given, ie, predicting flight delays. This should include some discussion of why this is an important task from a business perspective, who the stakeholders are, etc.. Some literature review will be helpful to figure out how this problem is being solved now, and the State Of The Art (SOTA) in this domain. Introduce the goal of your analysis. What questions will you seek to answer, why do people perform this kind of analysis on this kind of data? Preview what level of performance your model would need to achieve to be practically useful.

## Problem
The problem in front of us goes beyond just predicting flight delays. Each flight contains passengers that are trying to get to their next flight or to their final destination. A delay of one flight can have cascading delays to other flights. Flight delays also present an additional operational costs for the airlines, in particular if airlines have to reroute or refund passengers that have missed connections. Planes are far too expensive to just be discarded after they are used and extremely likely to be used again within the hour.  This means there is also an optimization problem to solve on how to efficiently re-route planes or make use of spare resources (planes and crews) to ensure that the next flight leaves on time.

The FAA defines a flight delay as a flight that is 15 minutes or more off schedule. This means that a flight can be late for either departure or arrive and still qualify as a late flight. We'll also need to predict the delay two hours before the flight would take off. This means that the weather data will be from 2 hours before the flight was intended to take off.

In [3]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/"))

path name size dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/_SUCCESS _SUCCESS 0 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00000-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00000-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 1030519 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00001-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00001-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 1161448 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00003-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00003-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 1109451 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00015-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00015-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 815185 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00020-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00020-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 586922 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00023-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00023-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 455595

In [4]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/"))

path name size dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/_SUCCESS _SUCCESS 0 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/_committed_6351095181992560034 _committed_6351095181992560034 6084 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/_started_6351095181992560034 _started_6351095181992560034 0 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00000-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3617-1-c000.snappy.parquet part-00000-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3617-1-c000.snappy.parquet 109256889 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00001-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3626-1-c000.snappy.parquet part-00001-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3626-1-c000.snappy.parquet 98375438 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00002-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3635-1-c000.snappy.parquet part-00002-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3635-1-c000.snappy.parquet 97627735 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00003-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3644-1-c000.snappy.parquet part-00003-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3644-1-c000.snappy.parquet 89677996 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00004-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3651-1-c000.snappy.parquet part-00004-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3651-1-c000.snappy.parquet 85150252 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00005-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3652-1-c000.snappy.parquet part-00005-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3652-1-c000.snappy.parquet 58641349 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00006-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3615-1-c000.snappy.parquet part-00006-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3615-1-c000.snappy.parquet 105917556 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00007-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3624-1-c000.snappy.parquet part-00007-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3624-1-c000.snappy.parquet 99676419 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00008-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3633-1-c000.snappy.parquet part-00008-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3633-1-c000.snappy.parquet 93829273 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00009-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3642-1-c000.snappy.parquet part-00009-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3642-1-c000.snappy.parquet 86008249 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00010-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3653-1-c000.snappy.parquet part-00010-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3653-1-c000.snappy.parquet 82240354 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00011-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3654-1-c000.snappy.parquet part-00011-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3654-1-c000.snappy.parquet 60657771 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00012-tid-6351095181992560034-563bd

In [5]:
# Libraries used
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from pyspark.sql.functions import lit
import datetime as datetime

In [6]:
##Start up Spark and SQL contexts
sc = spark.sparkContext
sqlContext = SQLContext(sc)
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.0 
 Master 
 spark://10.24.226.167:7077 
 AppName 
 Databricks Shell

# EDA & Discussion of Challenges
Determine 2-3 relevant EDA tasks that will help you make decisions about how you implement the algorithm to be scalable. Discuss any challenges that you anticipate based on the EDA you perform.

We have two datasets that we will leverage. One containing flight information and another with weather information. Let's start with breaking down the flight data. Our first task will be to create a binary column that contains whether the flight was delayed or not. 1 will represent that it delayed for either departure or arrival or both. 0 means that there is no delay.

There 109 total columns available in the flight data. Each provides information about the flight, but a number of these datum overlap with one another. To improve on the speed and efficiency of the data, we will need to do some dimension reduction to help eliminate highly correlated or useless columns. The dataset begins with 6 columns containing information relevant to the date of the flight. These are year, quarter, month, day_of_month, day_of_week, and fl_date.

## Explore Airlines Datasets

In [10]:
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/*.parquet")
display(airlines.sample(False, 0.0001, seed=4))

YEAR QUARTER MONTH DAY_OF_MONTH DAY_OF_WEEK FL_DATE OP_UNIQUE_CARRIER OP_CARRIER_AIRLINE_ID OP_CARRIER TAIL_NUM OP_CARRIER_FL_NUM ORIGIN_AIRPORT_ID ORIGIN_AIRPORT_SEQ_ID ORIGIN_CITY_MARKET_ID ORIGIN ORIGIN_CITY_NAME ORIGIN_STATE_ABR ORIGIN_STATE_FIPS ORIGIN_STATE_NM ORIGIN_WAC DEST_AIRPORT_ID DEST_AIRPORT_SEQ_ID DEST_CITY_MARKET_ID DEST DEST_CITY_NAME DEST_STATE_ABR DEST_STATE_FIPS DEST_STATE_NM DEST_WAC CRS_DEP_TIME DEP_TIME DEP_DELAY DEP_DELAY_NEW DEP_DEL15 DEP_DELAY_GROUP DEP_TIME_BLK TAXI_OUT WHEELS_OFF WHEELS_ON TAXI_IN CRS_ARR_TIME ARR_TIME ARR_DELAY ARR_DELAY_NEW ARR_DEL15 ARR_DELAY_GROUP ARR_TIME_BLK CANCELLED CANCELLATION_CODE DIVERTED CRS_ELAPSED_TIME ACTUAL_ELAPSED_TIME AIR_TIME FLIGHTS DISTANCE DISTANCE_GROUP CARRIER_DELAY WEATHER_DELAY NAS_DELAY SECURITY_DELAY LATE_AIRCRAFT_DELAY FIRST_DEP_TIME TOTAL_ADD_GTIME LONGEST_ADD_GTIME DIV_AIRPORT_LANDINGS DIV_REACHED_DEST DIV_ACTUAL_ELAPSED_TIME DIV_ARR_DELAY DIV_DISTANCE DIV1_AIRPORT DIV1_AIRPORT_ID DIV1_AIRPORT_SEQ_ID DIV1_WHEELS_ON DIV1_TOTAL_GTIME DIV1_LONGEST_GTIME DIV1_WHEELS_OFF DIV1_TAIL_NUM DIV2_AIRPORT DIV2_AIRPORT_ID DIV2_AIRPORT_SEQ_ID DIV2_WHEELS_ON DIV2_TOTAL_GTIME DIV2_LONGEST_GTIME DIV2_WHEELS_OFF DIV2_TAIL_NUM DIV3_AIRPORT DIV3_AIRPORT_ID DIV3_AIRPORT_SEQ_ID DIV3_WHEELS_ON DIV3_TOTAL_GTIME DIV3_LONGEST_GTIME DIV3_WHEELS_OFF DIV3_TAIL_NUM DIV4_AIRPORT DIV4_AIRPORT_ID DIV4_AIRPORT_SEQ_ID DIV4_WHEELS_ON DIV4_TOTAL_GTIME DIV4_LONGEST_GTIME DIV4_WHEELS_OFF DIV4_TAIL_NUM DIV5_AIRPORT DIV5_AIRPORT_ID DIV5_AIRPORT_SEQ_ID DIV5_WHEELS_ON DIV5_TOTAL_GTIME DIV5_LONGEST_GTIME DIV5_WHEELS_OFF DIV5_TAIL_NUM 2015 1 1 3 6 2015-01-03 OO 20304 OO N795SK 5242 13930 1393003 30977 ORD Chicago, IL IL 17 Illinois 41 11648 1164802 31648 FCA Kalispell, MT MT 30 Montana 84 945 956 11.0 11.0 0.0 0 0900-0959 40.0 1036 1251 5.0 1229 1256 27.0 27.0 1.0 1 1200-1259 0.0 null 0.0 224.0 240.0 195.0 1.0 1352.0 6 11.0 0.0 16.0 0.0 0.0 null null null 0 null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null 2015 1 1 26 1 2015-01-26 UA 19977 UA N847UA 606 13930 1393003 30977 ORD Chicago, IL IL 17 Illinois 41 14576 1457604 34576 ROC Rochester, NY NY 36 New York 22 1840 1923 43.0 43.0 1.0 2 1800-1859 20.0 1943 2156 3.0 2119 2159 40.0 40.0 1.0 2 2100-2159 0.0 null 0.0 99.0 96.0 73.0 1.0 528.0 3 0.0 40.0 0.0 0.0 0.0 null null null 0 null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null 2015 1 1 28 3 2015-01-28 DL 19790 DL N907DA 1534 10397 1039705 30397 ATL Atlanta, GA GA 13 Georgia 34 12339 1233904 32337 IND Indianapolis, IN IN 18 Indiana 42 1340 1339 -1.0 0.0 0.0 -1 1300-1359 9.0 1348 1503 4.0 1513 1507 -6.0 0.0 0.0 -1 1500-1559 0.0 null 0.0 93.0 88.0 75.0 1.0 432.0 2 null null null null null null null null 0 null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null 2015 1 3 27 5 2015-03-27 WN 19393 WN N604SW 1878 10397 1039705 30397 ATL Atlanta, GA GA 13 Georgia 34 14100 1410002 34100 PHL Philadelphia, PA PA 42 Pennsylvania 23 2200 2347 107.0 107.0 1.0 7 2200-2259 10.0 2357 119 4.0 10 123 73.0 73.0 1.0 4 0001-0559 0.0 null 0.0 130.0 96.0 82.0 1.0 666.0 3 72.0 0.0 0.0 0.0 1.0 null null null 0 null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null 2015 1 2 2 1 2015-02-02 DL 19790 DL N807DN 1271 10397 1039705 30397 ATL Atlanta, GA GA 13 Georgia 34 12451 1245102 31136 JAX Jacksonville, FL FL 12 Florida 33 1455 1455 0.0 0.0 0.0 0 1400-1459 12.0 1507 1548

In [11]:
## This will show the structure of the airline dataset
airlines.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: string (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
-- OP_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- OP_CARRIER_FL_NUM: integer (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- FIRST_DEP_TIME: integer (nullable = true)
-- TOTAL_ADD_GTIME: double (nullable = true)
-- LONGEST_ADD_GTIME: double (nullable = true)
-- DIV_AIRPORT_LANDINGS: integer (nullable = true)
-- DIV_REACHED_DEST: double (nullable = true)
-- DIV_ACTUAL_ELAPSED_TIME: double (nullable = true)
-- DIV_ARR_DELAY: double (nullable = true)
-- DIV_DISTANCE: double (nullable = true)
-- DIV1_AIRPORT: string (nullable = true)
-- DIV1_AIRPORT_ID: integer (nullable = true)
-- DIV1_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV1_WHEELS_ON: integer (nullable = true)
-- DIV1_TOTAL_GTIME: double (nullable = true)
-- DIV1_LONGEST_GTIME: double (nullable = true)
-- DIV1_WHEELS_OFF: integer (nullable = true)
-- DIV1_TAIL_NUM: string (nullable = true)
-- DIV2_AIRPORT: string (nullable = true)
-- DIV2_AIRPORT_ID: integer (nullable = true)
-- DIV2_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV2_WHEELS_ON: integer (nullable = true)
-- DIV2_TOTAL_GTIME: double (nullable = true)
-- DIV2_LONGEST_GTIME: double (nullable = true)
-- DIV2_WHEELS_OFF: integer (nullable = true)
-- DIV2_TAIL_NUM: string (nullable = true)
-- DIV3_AIRPORT: string (nullable = true)
-- DIV3_AIRPORT_ID: integer (nullable = true)
-- DIV3_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV3_WHEELS_ON: integer (nullable = true)
-- DIV3_TOTAL_GTIME: double (nullable = true)
-- DIV3_LONGEST_GTIME: double (nullable = true)
-- DIV3_WHEELS_

In [12]:
df_airlines = airlines.toPandas()

In [13]:
df_airlines.describe()

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN_STATE_FIPS 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST_STATE_FIPS 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 CANCELLED 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 
 
 
 
 count 
 161057.0 
 161057.0 
 161057.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 1.610570e+05 
 161057.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 1.610570e+05 
 161057.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 156231.000000 
 156231.000000 
 156231.000000 
 156231.000000 
 156231.000000 
 156121.000000 
 156121.000000 
 155957.000000 
 155957.000000 
 161057.000000 
 155957.000000 
 155692.000000 
 155692.000000 
 155692.000000 
 155692.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 155692.000000 
 155692.000000 
 161057.0 
 161057.000000 
 161057.000000 
 34915.000000 
 34915.000000 
 34915.00000 
 34915.000000 
 34915.000000 
 1053.000000 
 1053.000000 
 1053.000000 
 161057.000000 
 381.000000 
 265.000000 
 265.000000 
 381.000000 
 429.000000 
 4.290000e+02 
 429.000000 
 429.000000 
 429.000000 
 278.000000 
 17.000000 
 1.700000e+01 
 17.000000 
 17.000000 
 17.000000 
 5.000000 
 1.0 
 1.0 
 1.0 
 1.0 
 1.0 
 
 
 mean 
 2015.0 
 1.0 
 2.036552 
 15.591505 
 3.916520 
 20010.918153 
 2508.798711 
 11963.802592 
 1.196384e+06 
 30654.216389 
 14.773906 
 37.104336 
 12707.705111 
 1.270773e+06 
 32018.143148 
 28.618719 
 49.025475 
 1422.252768 
 1437.216897 
 12.583828 
 14.368870 
 0.234813 
 0.252427 
 18.062086 
 1465.591503 
 1563.578172 
 6.149086 
 1584.805361 
 1566.326064 
 7.511568 
 14.163483 
 0.224257 
 0.008729 
 0.030946 
 0.002366 
 124.434784 
 120.037144 
 95.836177 
 1.0 
 662.231340 
 3.124546 
 19.903050 
 6.769039 
 12.16838 
 0.017557 
 19.076672 
 1399.992403 
 36.226021 
 35.970560 
 0.005116 
 0.695538 
 347.101887 
 232.633962 
 47.908136 
 12499.242424 
 1.249927e+06 
 1464.046620 
 27.405594 
 22.242424 
 1474.884892 
 12674.176471 
 1.267421e+06 
 1341.411765 
 11.941176 
 10.235294 
 1392.200000 
 10397.0 
 1039705.0 
 51.0 
 4.0 
 4.0 
 
 
 std 
 0.0 
 0.0 
 0.832660 
 8.679528 
 1.995815 
 314.626497 
 1746.419264 
 1755.181564 
 1.755172e+05 
 288.141893 
 1.987185 
 3.477575 
 1522.429280 
 1.522428e+05 
 1388.348035 
 16.113962 
 21.488664 
 453.970474 
 463.857461 
 36.393693 
 35.596949 
 0.423883 
 2.142223 
 9.087123 
 465.871793 
 488.380807 
 4.818137 
 467.098093 
 494.175085 
 39.550690 
 36.196183 
 0.417093 
 2.305651 
 0.173171 
 0.048580 
 64.537094 
 63.288804 
 61.754708 
 0.0 
 474.807024 
 1.839999 
 38.098095 
 29.870353 
 25.07071 
 0.805233 
 36.269235 
 482.825093 
 28.256333 
 27.885904 
 0.163356 
 0.460784 
 204.471624 
 206.620475 
 124.824801 
 1657.100266 
 1.657094e+05 
 564.358918 
 25.880631 
 24.029032 
 543.504354 
 1805.210349 
 1.805201e+05 
 617.786073 
 9.222734 
 7.198345 
 586.312374 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 min 
 2015.0 
 1.0 
 1.000000 
 1.000000 
 1.000000 
 19393.000000 
 3.000000 
 10

In [14]:
# Unique value counts of each column
for col in df_airlines.columns:
  print(col, df_airlines[col].nunique())
  
# From DIV4 and DIV5 related columns do not have values, we could remove them

YEAR 1
QUARTER 1
MONTH 3
DAY_OF_MONTH 31
DAY_OF_WEEK 7
FL_DATE 90
OP_UNIQUE_CARRIER 13
OP_CARRIER_AIRLINE_ID 13
OP_CARRIER 13
TAIL_NUM 3907
OP_CARRIER_FL_NUM 3853
ORIGIN_AIRPORT_ID 2
ORIGIN_AIRPORT_SEQ_ID 2
ORIGIN_CITY_MARKET_ID 2
ORIGIN 2
ORIGIN_CITY_NAME 2
ORIGIN_STATE_ABR 2
ORIGIN_STATE_FIPS 2
ORIGIN_STATE_NM 2
ORIGIN_WAC 2
DEST_AIRPORT_ID 192
DEST_AIRPORT_SEQ_ID 192
DEST_CITY_MARKET_ID 175
DEST 192
DEST_CITY_NAME 188
DEST_STATE_ABR 51
DEST_STATE_FIPS 51
DEST_STATE_NM 51
DEST_WAC 51
CRS_DEP_TIME 978
DEP_TIME 1244
DEP_DELAY 473
DEP_DELAY_NEW 451
DEP_DEL15 2
DEP_DELAY_GROUP 15
DEP_TIME_BLK 19
TAXI_OUT 137
WHEELS_OFF 1259
WHEELS_ON 1279
TAXI_IN 111
CRS_ARR_TIME 1055
ARR_TIME 1281
ARR_DELAY 524
ARR_DELAY_NEW 461
ARR_DEL15 2
ARR_DELAY_GROUP 15
ARR_TIME_BLK 19
CANCELLED 2
CANCELLATION_CODE 3
DIVERTED 2
CRS_ELAPSED_TIME 316
ACTUAL_ELAPSED_TIME 480
AIR_TIME 450
FLIGHTS 1
DISTANCE 291
DISTANCE_GROUP 10
CARRIER_DELAY 315
WEATHER_DELAY 286
NAS_DELAY 250
SECURITY_DELAY 21
LATE_AIRCRAFT_DELAY 300
FIRST_DEP_TIME 650
TOTAL_ADD_GTIME 129
LONGEST_ADD_GTIME 126
DIV_AIRPORT_LANDINGS 4
DIV_REACHED_DEST 2
DIV_ACTUAL_ELAPSED_TIME 199
DIV_ARR_DELAY 181
DIV_DISTANCE 69
DIV1_AIRPORT 98
DIV1_AIRPORT_ID 98
DIV1_AIRPORT_SEQ_ID 98
DIV1_WHEELS_ON 348
DIV1_TOTAL_GTIME 93
DIV1_LONGEST_GTIME 80
DIV1_WHEELS_OFF 243
DIV1_TAIL_NUM 250
DIV2_AIRPORT 7
DIV2_AIRPORT_ID 7
DIV2_AIRPORT_SEQ_ID 7
DIV2_WHEELS_ON 17
DIV2_TOTAL_GTIME 10
DIV2_LONGEST_GTIME 10
DIV2_WHEELS_OFF 5
DIV2_TAIL_NUM 5
DIV3_AIRPORT 1
DIV3_AIRPORT_ID 1
DIV3_AIRPORT_SEQ_ID 1
DIV3_WHEELS_ON 1
DIV3_TOTAL_GTIME 1
DIV3_LONGEST_GTIME 1
DIV3_WHEELS_OFF 0
DIV3_TAIL_NUM 0
DIV4_AIRPORT 0
DIV4_AIRPORT_ID 0
DIV4_AIRPORT_SEQ_ID 0
DIV4_WHEELS_ON 0
DIV4_TOTAL_GTIME 0
DIV4_LONGEST_GTIME 0
DIV4_WHEELS_OFF 0
DIV4_TAIL_NUM 0
DIV5_AIRPORT 0
DIV5_AIRPORT_ID 0
DIV5_AIRPORT_SEQ_ID 0
DIV5_WHEELS_ON 0
DIV5_TOTAL_GTIME 0
DIV5_LONGEST_GTIME 0
DIV5_WHEELS_OFF 0
DIV5_TAIL_NUM 0

## Split the  Airlines dataset
- airlines_eda (5%)
- airlines_other (95%)

In [16]:
#repartition ensures that there is no possibility of overlap between data splits. See: https://medium.com/udemy-engineering/pyspark-under-the-hood-randomsplit-and-sample-inconsistencies-examined-7c6ec62644bc
airlines_eda, airlines_other = airlines.repartition(20,'FL_DATE').randomSplit([0.05,0.95], seed=4)
airlines_eda = airlines_eda.toPandas()
airlines_other = airlines_other.toPandas()

In [17]:
print("Row counts of airlines_eda: ", airlines_eda.count()[0])
print("Row counts of airlines_other: ", airlines_other.count()[0])

Row counts of airlines_eda: 8027
Row counts of airlines_other: 153030

## Explore airlines_eda Datasets

In [19]:
airlines_eda.head()

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN 
 ORIGIN_CITY_NAME 
 ORIGIN_STATE_ABR 
 ORIGIN_STATE_FIPS 
 ORIGIN_STATE_NM 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 ... 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 
 
 
 
 0 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N014AA 
 194 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 11298 
 1129803 
 30194 
 DFW 
 Dallas/Fort Worth, TX 
 TX 
 48 
 Texas 
 74 
 1525 
 1815.0 
 170.0 
 170.0 
 1.0 
 11.0 
 1500-1559 
 16.0 
 1831.0 
 1920.0 
 8.0 
 ... 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 1 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3HJAA 
 1142 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 11057 
 1105703 
 31057 
 CLT 
 Charlotte, NC 
 NC 
 37 
 North Carolina 
 36 
 1755 
 1814.0 
 19.0 
 19.0 
 1.0 
 1.0 
 1700-1759 
 19.0 
 1833.0 
 2103.0 
 10.0 
 ... 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 2 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3KPAA 
 1342 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 13303 
 1330303 
 32467 
 MIA 
 Miami, FL 
 FL 
 12 
 Florida 
 33 
 500 
 454.0 
 -6.0 
 0.0 
 0.0 
 -1.0 
 0001-0559 
 13.0 
 507.0 
 840.0 
 7.0 
 ... 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 3 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3LJAA 
 368 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 12953 
 1295302 
 31703 
 LGA 
 New York, NY 
 NY 
 36 
 New York 
 22 
 1320 
 1357.0 
 37.0 
 37.0 
 1.0 
 2.0 
 1300-1359 
 14.0 
 1411.0 
 1643.0 
 9.0 
 ... 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 4 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3LLAA 
 346 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 12953 
 1295302 
 31703 
 LGA 
 New York, NY 
 NY 
 36 
 New York 
 22 
 1915 
 1924.0 
 9.0 
 9.0 
 0.0 
 0.0 
 1900-1959 
 15.0 
 1939.0 
 2216.0 
 5.0 
 ... 
 None 
 NaN 
 NaN 
 NaN 


In [20]:
airlines_eda.describe()

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN_STATE_FIPS 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST_STATE_FIPS 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 CANCELLED 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 
 
 
 
 count 
 8027.0 
 8027.0 
 8027.000000 
 8027.000000 
 8027.000000 
 8027.000000 
 8027.000000 
 8027.000000 
 8.027000e+03 
 8027.000000 
 8027.000000 
 8027.000000 
 8027.000000 
 8.027000e+03 
 8027.000000 
 8027.000000 
 8027.000000 
 8027.000000 
 7810.000000 
 7810.000000 
 7810.000000 
 7810.000000 
 7810.000000 
 7805.000000 
 7805.000000 
 7799.000000 
 7799.000000 
 8027.00000 
 7799.000000 
 7788.000000 
 7788.000000 
 7788.000000 
 7788.000000 
 8027.000000 
 8027.000000 
 8027.000000 
 7788.000000 
 7788.000000 
 8027.0 
 8027.000000 
 8027.000000 
 1720.000000 
 1720.000000 
 1720.000000 
 1720.000000 
 1720.000000 
 42.000000 
 42.000000 
 42.000000 
 8027.000000 
 16.000000 
 11.000000 
 11.000000 
 16.000000 
 17.000000 
 1.700000e+01 
 17.000000 
 17.000000 
 17.000000 
 11.000000 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 
 
 mean 
 2015.0 
 1.0 
 2.032889 
 15.488850 
 3.936589 
 20011.542046 
 2524.316307 
 11980.181886 
 1.198022e+06 
 30656.905320 
 14.792450 
 37.136788 
 12699.552137 
 1.269958e+06 
 32034.491591 
 28.590881 
 49.091068 
 1424.371621 
 1439.079257 
 12.578873 
 14.375032 
 0.226504 
 0.240205 
 17.966047 
 1467.107880 
 1566.285037 
 6.102321 
 1585.73477 
 1566.981536 
 7.440036 
 14.217386 
 0.220853 
 -0.001798 
 0.027781 
 0.001993 
 124.046593 
 119.456343 
 95.387134 
 1.0 
 658.550392 
 3.112495 
 20.126744 
 6.391860 
 12.291860 
 0.010465 
 20.093023 
 1501.071429 
 36.023810 
 35.476190 
 0.003114 
 0.687500 
 385.727273 
 282.090909 
 13.000000 
 11848.823529 
 1.184885e+06 
 1402.941176 
 42.176471 
 37.235294 
 1365.363636 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 std 
 0.0 
 0.0 
 0.834656 
 8.658396 
 1.994096 
 317.153089 
 1761.485398 
 1757.071807 
 1.757062e+05 
 288.452207 
 1.989326 
 3.481320 
 1521.577603 
 1.521577e+05 
 1395.457743 
 16.043046 
 21.396295 
 453.810174 
 462.970942 
 37.377188 
 36.597891 
 0.418596 
 2.146968 
 9.045715 
 464.881805 
 485.486466 
 4.551300 
 466.26342 
 492.751888 
 40.634351 
 37.304909 
 0.414848 
 2.313882 
 0.164356 
 0.044604 
 64.039549 
 62.714678 
 61.419870 
 0.0 
 471.667951 
 1.824174 
 35.815963 
 29.610816 
 29.987621 
 0.323423 
 38.945489 
 511.957382 
 23.380718 
 23.194026 
 0.109891 
 0.478714 
 310.661903 
 308.357732 
 26.698315 
 1072.062687 
 1.072059e+05 
 561.201554 
 52.017107 
 50.568678 
 476.004469 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 min 
 2015.0 
 1.0 
 1.000000 
 1.000000 
 1.000000 
 19393.000000 
 7.000000 
 10397.000000 
 1.039705e+06 
 30397.000000 
 13.000000 
 34.000000 
 10135.000000 
 1.013503e+06 
 30135.000000 
 1.000000 
 1.000000 
 500.000000 
 3.000000 
 -16.000000 
 0.000000 
 0.000000 
 -2.000000 
 1.000000 
 2.000000 
 1.000000 

In [21]:
# Quick look at each of the columns data
airlines_eda.hist(figsize=(25,25), bins=110)

In [22]:
##Correlation Matrix
air_corr = airlines_eda.corr()
fig, ax = plt.subplots(figsize=(12, 12))
mask = np.zeros_like(air_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sn.diverging_palette(240, 10, as_cmap=True)
sn.heatmap(air_corr, mask=mask, cmap=cmap, center=0, linewidths=.5)
plt.title("Correlations between features.")
display(plt.show())

In [23]:
# Unique value counts of each column
for col in airlines_eda.columns:
  print(col, airlines_eda[col].nunique())

YEAR 1
QUARTER 1
MONTH 3
DAY_OF_MONTH 31
DAY_OF_WEEK 7
FL_DATE 90
OP_UNIQUE_CARRIER 13
OP_CARRIER_AIRLINE_ID 13
OP_CARRIER 13
TAIL_NUM 2573
OP_CARRIER_FL_NUM 2453
ORIGIN_AIRPORT_ID 2
ORIGIN_AIRPORT_SEQ_ID 2
ORIGIN_CITY_MARKET_ID 2
ORIGIN 2
ORIGIN_CITY_NAME 2
ORIGIN_STATE_ABR 2
ORIGIN_STATE_FIPS 2
ORIGIN_STATE_NM 2
ORIGIN_WAC 2
DEST_AIRPORT_ID 190
DEST_AIRPORT_SEQ_ID 190
DEST_CITY_MARKET_ID 173
DEST 190
DEST_CITY_NAME 186
DEST_STATE_ABR 51
DEST_STATE_FIPS 51
DEST_STATE_NM 51
DEST_WAC 51
CRS_DEP_TIME 786
DEP_TIME 1080
DEP_DELAY 241
DEP_DELAY_NEW 225
DEP_DEL15 2
DEP_DELAY_GROUP 15
DEP_TIME_BLK 19
TAXI_OUT 90
WHEELS_OFF 1085
WHEELS_ON 1077
TAXI_IN 55
CRS_ARR_TIME 979
ARR_TIME 1071
ARR_DELAY 279
ARR_DELAY_NEW 230
ARR_DEL15 2
ARR_DELAY_GROUP 15
ARR_TIME_BLK 19
CANCELLED 2
CANCELLATION_CODE 3
DIVERTED 2
CRS_ELAPSED_TIME 290
ACTUAL_ELAPSED_TIME 327
AIR_TIME 314
FLIGHTS 1
DISTANCE 283
DISTANCE_GROUP 10
CARRIER_DELAY 146
WEATHER_DELAY 88
NAS_DELAY 108
SECURITY_DELAY 3
LATE_AIRCRAFT_DELAY 154
FIRST_DEP_TIME 42
TOTAL_ADD_GTIME 30
LONGEST_ADD_GTIME 31
DIV_AIRPORT_LANDINGS 3
DIV_REACHED_DEST 2
DIV_ACTUAL_ELAPSED_TIME 11
DIV_ARR_DELAY 11
DIV_DISTANCE 6
DIV1_AIRPORT 13
DIV1_AIRPORT_ID 13
DIV1_AIRPORT_SEQ_ID 13
DIV1_WHEELS_ON 17
DIV1_TOTAL_GTIME 14
DIV1_LONGEST_GTIME 16
DIV1_WHEELS_OFF 11
DIV1_TAIL_NUM 10
DIV2_AIRPORT 0
DIV2_AIRPORT_ID 0
DIV2_AIRPORT_SEQ_ID 0
DIV2_WHEELS_ON 0
DIV2_TOTAL_GTIME 0
DIV2_LONGEST_GTIME 0
DIV2_WHEELS_OFF 0
DIV2_TAIL_NUM 0
DIV3_AIRPORT 0
DIV3_AIRPORT_ID 0
DIV3_AIRPORT_SEQ_ID 0
DIV3_WHEELS_ON 0
DIV3_TOTAL_GTIME 0
DIV3_LONGEST_GTIME 0
DIV3_WHEELS_OFF 0
DIV3_TAIL_NUM 0
DIV4_AIRPORT 0
DIV4_AIRPORT_ID 0
DIV4_AIRPORT_SEQ_ID 0
DIV4_WHEELS_ON 0
DIV4_TOTAL_GTIME 0
DIV4_LONGEST_GTIME 0
DIV4_WHEELS_OFF 0
DIV4_TAIL_NUM 0
DIV5_AIRPORT 0
DIV5_AIRPORT_ID 0
DIV5_AIRPORT_SEQ_ID 0
DIV5_WHEELS_ON 0
DIV5_TOTAL_GTIME 0
DIV5_LONGEST_GTIME 0
DIV5_WHEELS_OFF 0
DIV5_TAIL_NUM 0

In [24]:
# Check the depature airports
airlines_eda.ORIGIN.value_counts()

Out[18]: ATL 4430
ORD 3597
Name: ORIGIN, dtype: int64

In [25]:
# To check all the columns
airlines_eda.iloc[:, 0:40].head()

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN 
 ORIGIN_CITY_NAME 
 ORIGIN_STATE_ABR 
 ORIGIN_STATE_FIPS 
 ORIGIN_STATE_NM 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 
 
 
 
 0 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N014AA 
 194 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 11298 
 1129803 
 30194 
 DFW 
 Dallas/Fort Worth, TX 
 TX 
 48 
 Texas 
 74 
 1525 
 1815.0 
 170.0 
 170.0 
 1.0 
 11.0 
 1500-1559 
 16.0 
 1831.0 
 1920.0 
 8.0 
 
 
 1 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3HJAA 
 1142 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 11057 
 1105703 
 31057 
 CLT 
 Charlotte, NC 
 NC 
 37 
 North Carolina 
 36 
 1755 
 1814.0 
 19.0 
 19.0 
 1.0 
 1.0 
 1700-1759 
 19.0 
 1833.0 
 2103.0 
 10.0 
 
 
 2 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3KPAA 
 1342 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 13303 
 1330303 
 32467 
 MIA 
 Miami, FL 
 FL 
 12 
 Florida 
 33 
 500 
 454.0 
 -6.0 
 0.0 
 0.0 
 -1.0 
 0001-0559 
 13.0 
 507.0 
 840.0 
 7.0 
 
 
 3 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3LJAA 
 368 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 12953 
 1295302 
 31703 
 LGA 
 New York, NY 
 NY 
 36 
 New York 
 22 
 1320 
 1357.0 
 37.0 
 37.0 
 1.0 
 2.0 
 1300-1359 
 14.0 
 1411.0 
 1643.0 
 9.0 
 
 
 4 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3LLAA 
 346 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 12953 
 1295302 
 31703 
 LGA 
 New York, NY 
 NY 
 36 
 New York 
 22 
 1915 
 1924.0 
 9.0 
 9.0 
 0.0 
 0.0 
 1900-1959 
 15.0 
 1939.0 
 2216.0 
 5.0

In [26]:
airlines_eda.iloc[:, 40:80].head()

CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 ARR_TIME_BLK 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 
 
 
 
 0 
 1650 
 1928.0 
 158.0 
 158.0 
 1.0 
 10.0 
 1600-1659 
 0.0 
 None 
 0.0 
 145.0 
 133.0 
 109.0 
 1.0 
 731.0 
 3 
 0.0 
 0.0 
 0.0 
 0.0 
 158.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 
 
 1 
 2055 
 2113.0 
 18.0 
 18.0 
 1.0 
 1.0 
 2000-2059 
 0.0 
 None 
 0.0 
 120.0 
 119.0 
 90.0 
 1.0 
 599.0 
 3 
 0.0 
 0.0 
 0.0 
 0.0 
 18.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 
 
 2 
 859 
 847.0 
 -12.0 
 0.0 
 0.0 
 -1.0 
 0800-0859 
 0.0 
 None 
 0.0 
 179.0 
 173.0 
 153.0 
 1.0 
 1197.0 
 5 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 
 
 3 
 1620 
 1652.0 
 32.0 
 32.0 
 1.0 
 2.0 
 1600-1659 
 0.0 
 None 
 0.0 
 120.0 
 115.0 
 92.0 
 1.0 
 733.0 
 3 
 0.0 
 0.0 
 0.0 
 0.0 
 32.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 
 
 4 
 2215 
 2221.0 
 6.0 
 6.0 
 0.0 
 0.0 
 2200-2259 
 0.0 
 None 
 0.0 
 120.0 
 117.0 
 97.0 
 1.0 
 733.0 
 3 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN

In [27]:
airlines_eda.iloc[:, 80:].head()

DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 
 
 
 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 1 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 2 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 3 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 4 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None

#### Discussion of flight data variables

* Exclusion
  * The variables beginning with DIV2, DIV3, DIV4, and DIV5 are completely empty
  * All variables with SEQ_ID contain unique identifiers at a given point in time. For our purposes, this is less useful and redundant to the more generalizable fields with the same information applicable across time
  * ORIGIN, ORIGIN_CITY_NAME, ORIGIN_STATE_FIPS, ORIGIN_STATE_NM and the corresponding fields for DEST are all redundant with other fields that we will retain that identify airport ID, city, market and state

* Future Notes
  * There is a good amount of redundant data representation in Departure time and arrival time fields, though not sure which represention we will need yet, so leaving them in. Can revisit their exclusion later.

## Explore Weather Datasets

In [30]:
weather = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/*.parquet")
#only Q1 2015 for EDA
weather = weather.filter(weather["DATE"]<(lit("2015-04-01"))) 

In [31]:
weather.count()

Out[23]: 29643209

In [32]:
weather.printSchema()

root
-- STATION: string (nullable = true)
-- DATE: timestamp (nullable = true)
-- SOURCE: short (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)
-- ELEVATION: double (nullable = true)
-- NAME: string (nullable = true)
-- REPORT_TYPE: string (nullable = true)
-- CALL_SIGN: string (nullable = true)
-- QUALITY_CONTROL: string (nullable = true)
-- WND: string (nullable = true)
-- CIG: string (nullable = true)
-- VIS: string (nullable = true)
-- TMP: string (nullable = true)
-- DEW: string (nullable = true)
-- SLP: string (nullable = true)
-- AW1: string (nullable = true)
-- GA1: string (nullable = true)
-- GA2: string (nullable = true)
-- GA3: string (nullable = true)
-- GA4: string (nullable = true)
-- GE1: string (nullable = true)
-- GF1: string (nullable = true)
-- KA1: string (nullable = true)
-- KA2: string (nullable = true)
-- MA1: string (nullable = true)
-- MD1: string (nullable = true)
-- MW1: string (nullable = true)
-- MW2: string (nullable = true)
-- OC1: string (nullable = true)
-- OD1: string (nullable = true)
-- OD2: string (nullable = true)
-- REM: string (nullable = true)
-- EQD: string (nullable = true)
-- AW2: string (nullable = true)
-- AX4: string (nullable = true)
-- GD1: string (nullable = true)
-- AW5: string (nullable = true)
-- GN1: string (nullable = true)
-- AJ1: string (nullable = true)
-- AW3: string (nullable = true)
-- MK1: string (nullable = true)
-- KA4: string (nullable = true)
-- GG3: string (nullable = true)
-- AN1: string (nullable = true)
-- RH1: string (nullable = true)
-- AU5: string (nullable = true)
-- HL1: string (nullable = true)
-- OB1: string (nullable = true)
-- AT8: string (nullable = true)
-- AW7: string (nullable = true)
-- AZ1: string (nullable = true)
-- CH1: string (nullable = true)
-- RH3: string (nullable = true)
-- GK1: string (nullable = true)
-- IB1: string (nullable = true)
-- AX1: string (nullable = true)
-- CT1: string (nullable = true)
-- AK1: string (nullable = true)
-- CN2: string (nullable = true)
-- OE1: string (nullable = true)
-- MW5: string (nullable = true)
-- AO1: string (nullable = true)
-- KA3: string (nullable = true)
-- AA3: string (nullable = true)
-- CR1: string (nullable = true)
-- CF2: string (nullable = true)
-- KB2: string (nullable = true)
-- GM1: string (nullable = true)
-- AT5: string (nullable = true)
-- AY2: string (nullable = true)
-- MW6: string (nullable = true)
-- MG1: string (nullable = true)
-- AH6: string (nullable = true)
-- AU2: string (nullable = true)
-- GD2: string (nullable = true)
-- AW4: string (nullable = true)
-- MF1: string (nullable = true)
-- AA1: string (nullable = true)
-- AH2: string (nullable = true)
-- AH3: string (nullable = true)
-- OE3: string (nullable = true)
-- AT6: string (nullable = true)
-- AL2: string (nullable = true)
-- AL3: string (nullable = true)
-- AX5: string (nullable = true)
-- IB2: string (nullable = true)
-- AI3: string (nullable = true)
-- CV3: string (nullable = true)
-- WA1: string (nullable = true)
-- GH1: string (nullable = true)
-- KF1: string (nullable = true)
-- CU2: string (nullable = true)
-- CT3: string (nullable = true)
-- SA1: string (nullable = true)
-- AU1: string (nullable = true)
-- KD2: string (nullable = true)
-- AI5: string (nullable = true)
-- GO1: string (nullable = true)
-- GD3: string (nullable = true)
-- CG3: string (nullable = true)
-- AI1: string (nullable = true)
-- AL1: string (nullable = true)
-- AW6: string (nullable = true)
-- MW4: string (nullable = true)
-- AX6: string (nullable = true)
-- CV1: string (nullable = true)
-- ME1: string (nullable = true)
-- KC2: string (nullable = true)
-- CN1: string (nullable = true)
-- UA1: string (nullable = true)
-- GD5: string (nullable = true)
-- UG2: string (nullable = true)
-- AT3: string (nullable = true)
-- AT4: string (nullable = true)
-- GJ1: string (nullable = true)
-- MV1: string (nullable = true)
-- GA5: string (nullable = true)
-- CT2: string (nullable = true)
-- CG2: string

## Split the Weather dataset
- weather_eda (5%)
- weather_other (95%)

In [34]:
weather_eda, weather_other = weather.repartition(20,'DATE').randomSplit([0.05,0.95], seed=4)
weather_eda = weather_eda.toPandas()

In [35]:
weather_eda.shape

Out[26]: (1482291, 177)

In [36]:
weather_eda.head()

STATION 
 DATE 
 SOURCE 
 LATITUDE 
 LONGITUDE 
 ELEVATION 
 NAME 
 REPORT_TYPE 
 CALL_SIGN 
 QUALITY_CONTROL 
 WND 
 CIG 
 VIS 
 TMP 
 DEW 
 SLP 
 AW1 
 GA1 
 GA2 
 GA3 
 GA4 
 GE1 
 GF1 
 KA1 
 KA2 
 MA1 
 MD1 
 MW1 
 MW2 
 OC1 
 OD1 
 OD2 
 REM 
 EQD 
 AW2 
 AX4 
 GD1 
 AW5 
 GN1 
 AJ1 
 ... 
 GA6 
 KG1 
 AU3 
 AT7 
 KD1 
 GL1 
 IA1 
 GG2 
 OD3 
 UG1 
 CB1 
 AI6 
 CI1 
 CV2 
 AZ2 
 AD1 
 AH1 
 WD1 
 AA4 
 KC1 
 IA2 
 CF3 
 AI2 
 AT1 
 GD4 
 AX3 
 AH4 
 KB3 
 CU1 
 CN4 
 AT2 
 CG1 
 CF1 
 GG1 
 MV2 
 CW1 
 GG4 
 AB1 
 AH5 
 CN3 
 
 
 
 
 0 
 None 
 2015-01-01 10:30:00 
 6.0 
 40.98300 
 -122.69400 
 728.2 
 TRINITY CENTER AIRPORT, CA US 
 FM-15 
 KO86 
 V020 
 999,9,C,0000,5 
 22000,5,9,N 
 016093,5,N,5 
 -0040,5 
 -0070,5 
 99999,9 
 
 00,5,+99999,9,99,9 
 
 
 
 
 
 
 
 10217,5,09366,5 
 
 
 
 
 
 
 MET07501/01/15 02:30:02 METAR KO86 011030Z 000... 
 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 None 
 2015-01-01 16:35:00 
 6.0 
 37.60056 
 -81.55917 
 543.5 
 PINEVILLE KEE FIELD AIRPORT, WV US 
 FM-15 
 KI16 
 V020 
 999,9,9,9999,9 
 01676,5,M,N 
 016093,5,N,5 
 +0002,5 
 -0054,5 
 99999,9 
 
 08,5,+01676,5,99,9 
 
 
 
 
 
 
 
 10261,5,09618,5 
 
 
 
 
 
 
 MET08701/01/15 11:35:02 METAR KI16 011635Z ... 
 
 
 
 4,99,1,+01676,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 None 
 2015-01-01 20:35:00 
 6.0 
 30.21889 
 -96.37417 
 93.9 
 BRENHAM MUNICIPAL AIRPORT, TX US 
 FM-15 
 K11R 
 V020 
 360,5,N,0026,5 
 00152,5,M,N 
 004023,5,N,5 
 +0066,5 
 +0061,5 
 99999,9 
 51,5 
 08,5,+00152,5,99,9 
 
 
 
 
 
 
 
 10213,5,10100,5 
 
 
 
 
 
 
 MET09301/01/15 14:35:02 METAR K11R 012035Z 360... 
 
 
 
 4,99,1,+00152,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 None 
 2015-01-01 20:35:00 
 6.0 
 36.69850 
 -93.40220 
 411.2 
 BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, ... 
 FM-15 
 KFWB 
 V020 
 999,9,C,0000,5 
 01829,5,M,N 
 008047,5,N,5 
 +0020,5 
 -0020,5 
 99999,9 
 61,5 
 02,5,+01219,5,99,9 
 04,5,+01402,5,99,9 
 08,5,+01829,5,99,9 
 
 
 
 
 
 10234,5,09745,5 
 
 
 
 
 
 
 MET09401/01/15 14:35:02 METAR KFWB 012035Z 000... 
 
 
 
 1,99,1,+01219,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4 
 None 
 2015-01-01 20:35:00 
 6.0 
 37.60056 
 -81.55917 
 543.5 
 PINEVILLE KEE FIELD AIRPORT, WV US 
 FM-15 
 KI16 
 V020 
 999,9,9,9999,9 
 01341,5,M,N 
 016093,5,N,5 
 +0036,5 
 -0046,5 
 99999,9 
 
 08,5,+01341,5,99,9 
 
 
 
 
 
 
 
 10227,5,09586,5 
 
 
 
 
 
 
 MET08701/01/15 15:35:02 METAR KI16 012035Z ... 
 
 
 
 4,99,1,+01341,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 rows × 177 columns

In [37]:
#number of unique weather reporting stations
len(weather_eda.NAME.unique())

Out[28]: 11760

In [38]:
weather_eda.hist(figsize=(25,25), bins=110)

## Explore Station Datasets

In [40]:
stations = spark.read.option("header", "true").csv("dbfs:/mnt/mids-w261/data/DEMO8/gsod/stations.csv.gz")

In [41]:
stations = stations.toPandas()

In [42]:
stations.head()

usaf 
 wban 
 name 
 country 
 state 
 call 
 lat 
 lon 
 elev 
 begin 
 end 
 
 
 
 
 0 
 007018 
 99999 
 WXPOD 7018 
 None 
 None 
 None 
 0 
 0 
 +7018.0 
 20110309 
 20130730 
 
 
 1 
 007026 
 99999 
 WXPOD 7026 
 AF 
 None 
 None 
 0 
 0 
 +7026.0 
 20120713 
 20170822 
 
 
 2 
 007070 
 99999 
 WXPOD 7070 
 AF 
 None 
 None 
 0 
 0 
 +7070.0 
 20140923 
 20150926 
 
 
 3 
 008268 
 99999 
 WXPOD8278 
 AF 
 None 
 None 
 32.95 
 65.567 
 +1156.7 
 20100519 
 20120323 
 
 
 4 
 008307 
 99999 
 WXPOD 8318 
 AF 
 None 
 None 
 0 
 0 
 +8318.0 
 20100421 
 20100421

In [43]:
stations[stations['state']=='IL']

usaf 
 wban 
 name 
 country 
 state 
 call 
 lat 
 lon 
 elev 
 begin 
 end 
 
 
 
 
 6364 
 720103 
 99999 
 WILMETTE MARINES 
 US 
 IL 
 None 
 42.07 
 -87.68 
 +0183.0 
 19960101 
 19960214 
 
 
 6367 
 720141 
 04868 
 MARSHALL CO 
 US 
 IL 
 KC75 
 41.019 
 -89.386 
 +0173.1 
 20060101 
 20130430 
 
 
 6770 
 722417 
 99999 
 WILMETTE (CGS) 
 US 
 IL 
 None 
 42.067 
 -87.683 
 +0183.0 
 19870501 
 19910207 
 
 
 6771 
 722418 
 99999 
 CALUMET (CGS) 
 US 
 IL 
 63G 
 41.717 
 -87.533 
 +0180.0 
 19870501 
 19900810 
 
 
 7000 
 724337 
 03838 
 VANDALIA MUNICIPAL 
 US 
 IL 
 KVLA 
 39 
 -89.167 
 +0164.0 
 19730101 
 19730515 
 
 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 
 
 29125 
 744665 
 04838 
 PALWAUKEE MUNICIPAL ARPT 
 US 
 IL 
 KPWK 
 42.121 
 -87.905 
 +0193.9 
 20060101 
 20190304 
 
 
 29126 
 744666 
 53944 
 JACKSONVILLE MUNICIPAL AIRPOR 
 US 
 IL 
 KIJX 
 39.78 
 -90.238 
 +0190.2 
 20060101 
 20190304 
 
 
 29127 
 744672 
 04862 
 LOGAN COUNTY AIRPORT 
 US 
 IL 
 KAAA 
 40.158 
 -89.335 
 +0182.0 
 20060101 
 20190304 
 
 
 29228 
 999999 
 93816 
 EFFINGHAM COUNTY MEMORIAL ARP 
 US 
 IL 
 None 
 39.07 
 -88.533 
 +0178.9 
 19481201 
 20190304 
 
 
 29237 
 A00017 
 04868 
 MARSHALL COUNTY AIRPORT 
 US 
 IL 
 KC75 
 41.019 
 -89.386 
 +0173.1 
 20060101 
 20190304 
 
 
 
 150 rows × 11 columns

In [44]:
#weather_eda.join(stations, (weather_eda["CALL_SIGN"] == stations["call"]))

### Kyle's note:
- For the sample datasets train and test splits are fine. Make sure that when you are working in the **full dataset you split into train/dev/test**
- Be careful when it comes to feature reduction. In particular, when you have **correlated features**, you can create a new column by summing the original features. 
- This is a classification task - so while you may want to explore it as a regression problem, remeber that we are looking for a binary outcome: delay or no delay.
- Make sure you are considering all delays 15 minutes or longer. not just the DEP_DEL15 field. You will need to generate a new outcome variable.
- Please expand on your KNN matching technique.
- It's great that you are considering a multiclass model. Do make sure that you also report result for a binary dely vs no delay model.
- It's great that you are keeping a detailed report of your progress. For the weekly summaries, please just post a paragraph in the text box.
- Make sure you are considering all delays 15 minutes or longer. not just the DEP_DEL15 field. You will need to generate a new outcome variable.
- Please keep the weekly summaries short so they fit in the text box in the form. You can keep your own more detailed record to use by your team if you find that helpful.
- Make sure you are considerering all delays 15 minutes or longer. **not just the DEP_DEL15** field. You will need to **generate a new outcome variable**.
- How will you calculate RMSE for logistic regression? Or should you think about another evaluation metric?
- Keep up the good work!

# Feature Engineering
Apply relevant feature transformations, dimensionality reduction if needed, interaction terms, treatment of categorical variables, etc.. Justify your choices.
- What are the proposed features, and how are they derived from underlying data? 
- Do you need any dimensionality reduction? (ie L1, forward/backward selection, PCA, etc..)
- Specify the feature transformations for the pipeline and justify these features given the target (ie, hashing trick, tf-idf, stopword removal, lemmatization, tokenization, etc..)
- Other feature engineering efforts, ie interaction terms, Brieman’s method, etc…)

### 1. Create a new out come variable "DELAY_15"
- 1: Departure delay over 15 min
- 0: Departure not delay or delay less than 15 min

#### 1-1. First set the DELAY_15 as DEP_DEL15, change NaN value to 0

In [49]:
# Total counts of airlines is 161057
df_airlines.count()[0]

Out[34]: 161057

In [50]:
# There are NaN in DEP_DEL15. 119546 + 36685 != 161057
df_airlines.DEP_DEL15.value_counts()

Out[35]: 0.0 119546
1.0 36685
Name: DEP_DEL15, dtype: int64

In [51]:
# DEP_DEL15: Fill in O in NaN value
df_airlines['DEP_DEL15'] = df_airlines['DEP_DEL15'].fillna(0)
df_airlines.DEP_DEL15.value_counts()

Out[36]: 0.0 124372
1.0 36685
Name: DEP_DEL15, dtype: int64

In [52]:
# Assign the DELAY_15 as DEP_DEL15
df_airlines['DELAY_15'] = df_airlines.DEP_DEL15
print(df_airlines.DELAY_15.value_counts())
df_airlines.head()

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN 
 ORIGIN_CITY_NAME 
 ORIGIN_STATE_ABR 
 ORIGIN_STATE_FIPS 
 ORIGIN_STATE_NM 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015 
 1 
 1 
 1 
 4 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1040.0 
 5.0 
 5.0 
 0.0 
 0.0 
 1000-1059 
 15.0 
 1055.0 
 1419.0 
 6.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015 
 1 
 1 
 2 
 5 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1051.0 
 16.0 
 16.0 
 1.0 
 1.0 
 1000-1059 
 9.0 
 1100.0 
 1432.0 
 5.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015 
 1 
 1 
 3 
 6 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1041.0 
 6.0 
 6.0 
 0.0 
 0.0 
 1000-1059 
 22.0 
 1103.0 
 1437.0 
 20.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015 
 1 
 1 
 4 
 7 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1107.0 
 32.0 
 32.0 
 1.0 
 2.0 
 1000-1059 
 23.0 
 1130.0 
 1519.0 
 4.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1058.0 
 23.0 
 23.0 
 1.0 
 1.0 
 1000-1059 
 21.0 
 1119.0 
 1444.0 
 5.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 

#### 1-2. Handle exception for DELAY_15 in Canceled flight

In [54]:
# Check the canceled flight count
df_airlines.CANCELLED.value_counts()
# 218 canceled flights

Out[38]: 0.0 156073
1.0 4984
Name: CANCELLED, dtype: int64

In [55]:
# Get the rows of canceled flights
cancel_index = df_airlines.loc[df_airlines['CANCELLED'] == 1].index

In [56]:
# Some canceled flights had been labeled as delay flights, which should be labeled as non-delay flights
df_airlines.iloc[cancel_index,-1].value_counts()

Out[40]: 0.0 4882
1.0 102
Name: DELAY_15, dtype: int64

In [57]:
# When the flight is canceled assign DELAY_15 as 0
df_airlines.iloc[cancel_index,-1] = 0
df_airlines.iloc[cancel_index,-1].value_counts()

Out[41]: 0.0 4984
Name: DELAY_15, dtype: int64

#### 1-3. Check if there is an exception in DELAY_15 at diverted flight
Divert flight affected the arrivial time not departure time

In [59]:
# Check the canceled flight count
df_airlines.DIVERTED.value_counts()
# 25 diverted flights

Out[42]: 0.0 160676
1.0 381
Name: DIVERTED, dtype: int64

In [60]:
# Diverted flights does not need to change the values of DELAY_15, the values is same as DEP_DEL15
df_airlines.loc[df_airlines['DIVERTED'] == 1]

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN 
 ORIGIN_CITY_NAME 
 ORIGIN_STATE_ABR 
 ORIGIN_STATE_FIPS 
 ORIGIN_STATE_NM 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 3523 
 2015 
 1 
 1 
 26 
 1 
 2015-01-26 
 DL 
 19790 
 DL 
 N826DN 
 1865 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 13303 
 1330303 
 32467 
 MIA 
 Miami, FL 
 FL 
 12 
 Florida 
 33 
 955 
 958.0 
 3.0 
 3.0 
 0.0 
 0.0 
 0900-0959 
 20.0 
 1018.0 
 1342.0 
 5.0 
 ... 
 14027.0 
 1402702.0 
 1209.0 
 17.0 
 13.0 
 1317.0 
 N826DN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 4109 
 2015 
 1 
 1 
 31 
 6 
 2015-01-31 
 DL 
 19790 
 DL 
 N6713Y 
 1172 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 11503 
 1150303 
 31503 
 EGE 
 Eagle, CO 
 CO 
 8 
 Colorado 
 82 
 1015 
 1013.0 
 -2.0 
 0.0 
 0.0 
 -1.0 
 1000-1059 
 15.0 
 1028.0 
 1418.0 
 4.0 
 ... 
 11292.0 
 1129202.0 
 1229.0 
 38.0 
 34.0 
 1346.0 
 N6713Y 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 4257 
 2015 
 1 
 1 
 31 
 6 
 2015-01-31 
 DL 
 19790 
 DL 
 N6704Z 
 1757 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 14908 
 1490803 
 32575 
 SNA 
 Santa Ana, CA 
 CA 
 6 
 California 
 91 
 956 
 1001.0 
 5.0 
 5.0 
 0.0 
 0.0 
 0900-0959 
 14.0 
 1015.0 
 1457.0 
 4.0 
 ... 
 12892.0 
 1289203.0 
 1142.0 
 30.0 
 21.0 
 1434.0 
 N6704Z 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 4482 
 2015 
 1 
 1 
 3 
 6 
 2015-01-03 
 EV 
 20366 
 EV 
 N931EV 
 5143 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 14574 
 1457402 
 34574 
 ROA 
 Roanoke, VA 
 VA 
 51 
 Virginia 
 38 
 905 
 900.0 
 -5.0 
 0.0 
 0.0 
 -1.0 
 0900-0959 
 27.0 
 927.0 
 NaN 
 NaN 
 ... 
 10397.0 
 1039705.0 
 1016.0 
 21.0 
 13.0 
 1205.0 
 N931EV 
 GSO 
 11995.0 
 1199502.0 
 1345.0 
 3.0 
 3.0 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 4761 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 EV 
 20366 
 EV 
 N686BR 
 5247 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 10781 
 1078102 
 30781 
 BTR 
 Baton Rouge, LA 
 LA 
 22 
 Louisiana 
 72 
 2149 
 2146.0 
 -3.0 
 0.0 
 0.0 
 -1

The new outcame variables is "DELAY_15"
- 1: Departure delay over 15 min
- 0: Departure not delay or delay less than 15 min

### 2. Feature reduction: remove highly correlated variables and redundant variables
Flight variables document: https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236

#### 2-1. Remove redundant variables

##### Time period variables: 
- YEAR	
- QUARTER	
- MONTH	
- DAY_OF_MONTH	
- DAY_OF_WEEK	
- **FL_DATE**

These variables all represent the time period, so we can choose only FL_DATE variable to represent all these time period variables.

In [65]:
# delete time period redundant variables
del_list_time_period = ['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK']

for col in del_list_time_period:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN 
 ORIGIN_CITY_NAME 
 ORIGIN_STATE_ABR 
 ORIGIN_STATE_FIPS 
 ORIGIN_STATE_NM 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1040.0 
 5.0 
 5.0 
 0.0 
 0.0 
 1000-1059 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 10.0 
 0.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1051.0 
 16.0 
 16.0 
 1.0 
 1.0 
 1000-1059 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 22.0 
 1.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1041.0 
 6.0 
 6.0 
 0.0 
 0.0 
 1000-1059 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 42.0 
 1.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1107.0 
 32.0 
 32.0 
 1.0 
 2.0 
 1000-1059 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 68.0 
 1.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1058.0 
 23.0 
 23.0 
 1.0 
 1.0 
 1000-1059 
 21.0 
 1119.0 
 1444.0 
 5.0 


##### Origin related variables: 
- ORIGIN_AIRPORT_ID	
- ORIGIN_AIRPORT_SEQ_ID	
- ORIGIN_CITY_MARKET_ID	
- **ORIGIN**	
- ORIGIN_CITY_NAME	
- ORIGIN_STATE_ABR	
- ORIGIN_STATE_FIPS	
- ORIGIN_STATE_NM	
- ORIGIN_WAC

These variables all represent the origin airport, so we can choose only ORIGIN variable to represent all these origin related variables.

In [67]:
# delete origin related redundant variables
del_list_origin = ['ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS', 'ORIGIN_STATE_NM', 'ORIGIN_WAC']

for col in del_list_origin:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 ARR_TIME_BLK 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1040.0 
 5.0 
 5.0 
 0.0 
 0.0 
 1000-1059 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 10.0 
 0.0 
 0.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1051.0 
 16.0 
 16.0 
 1.0 
 1.0 
 1000-1059 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 22.0 
 1.0 
 1.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1041.0 
 6.0 
 6.0 
 0.0 
 0.0 
 1000-1059 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 42.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1107.0 
 32.0 
 32.0 
 1.0 
 2.0 
 1000-1059 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 68.0 
 1.0 
 4.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1058.0 
 23.0 
 23.0 
 1.0 
 1.0 
 1000-1059 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 34.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 ... 
 NaN 
 NaN 


##### Destination related variables: 
- DEST_AIRPORT_ID	
- DEST_AIRPORT_SEQ_ID	
- DEST_CITY_MARKET_ID	
- **DEST**	
- DEST_CITY_NAME	
- DEST_STATE_ABR	
- DEST_STATE_FIPS	
- DEST_STATE_NM	
- DEST_WAC	

These variables all represent the destination airport, so we can choose only DEST variable to represent all these destination related variables.

In [69]:
# delete destination related redundant variables
del_list_dest = ['DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST_CITY_NAME', 'DEST_STATE_ABR', 'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_WAC']

for col in del_list_dest:
  del df_airlines[col]
 
df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 ARR_TIME_BLK 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 5.0 
 5.0 
 0.0 
 0.0 
 1000-1059 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 10.0 
 0.0 
 0.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 5 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 16.0 
 16.0 
 1.0 
 1.0 
 1000-1059 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 22.0 
 1.0 
 1.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 5 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 6.0 
 6.0 
 0.0 
 0.0 
 1000-1059 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 42.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 5 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 32.0 
 32.0 
 1.0 
 2.0 
 1000-1059 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 68.0 
 1.0 
 4.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 5 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 23.0 
 23.0 
 1.0 
 1.0 
 1000-1059 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 34.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 5 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN

##### Departure Performance related variables:
1) 
- DEP_DELAY	
- **DEP_DELAY_NEW**
- DEP_DEL15	
- DEP_DELAY_GROUP

DEP_DELAY_NEW changes the early departure which is the negetive value in DEP_DELAY to 0. DEP_DEL15 is indicate if the delay is over 15 mins. DEP_DELAY_GROUP categorize the degree of DEP_DELAY into different groups. So we can use DEP_DELAY_NEW to represent these variables.

2)
- **CRS_DEP_TIME**
- DEP_TIME_BLK

DEP_TIME_BLK is changing CRS_DEP_TIME to hourly intervals.
So we can use CRS_DEP_TIME to represent these variables.

In [71]:
# delete departure performance related redundant variables
del_list_dest_perf = ['DEP_DELAY', 'DEP_DEL15', 'DEP_DELAY_GROUP', 'DEP_TIME_BLK']

for col in del_list_dest_perf:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY_NEW 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 ARR_TIME_BLK 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 5.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 10.0 
 0.0 
 0.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 5 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 16.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 22.0 
 1.0 
 1.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 5 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 6.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 42.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 5 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 32.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 68.0 
 1.0 
 4.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 5 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 23.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 34.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 5 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 


##### Arrival Performance related variables:

1) 
- ARR_DELAY	
- **ARR_DELAY_NEW**
- ARR_DEL15	
- ARR_DELAY_GROUP

ARR_DELAY_NEW changes the early departure which is the negative value in ARR_DELAY to 0. ARR_DEL15 is indicate if the delay is over 15 mins. ARR_DELAY_GROUP categorize the degree of ARR_DELAY into different groups. So we can use ARR_DELAY_NEW to represent these variables.

2)
- **CRS_ARR_TIME**
- ARR_TIME_BLK

ARR_TIME_BLK is changing CRS_ARR_TIME to hourly intervals.
So we can use CRS_ARR_TIME to represent these variables.

In [73]:
# delete arrival performance related redundant variables
del_list_arr_perf = ['ARR_DELAY', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK']

for col in del_list_arr_perf:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY_NEW 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY_NEW 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 5.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 5 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 16.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 5 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 6.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 5 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 32.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 5 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 23.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 5 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 N

##### Flight summaries
- **DISTANCE**	
- DISTANCE_GROUP

DISTANCE_GROUP is changing DISTANCE to distance Intervals, every 250 Miles, for Flight Segment. So we can use DISTANCE to represent these variables.

In [75]:
# delete flight summaries related redundant variables
del df_airlines['DISTANCE_GROUP']

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY_NEW 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY_NEW 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 5.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 16.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 6.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 32.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 23.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 Non

##### Diverted Airport Information
- DIV4_AIRPORT	
- DIV4_AIRPORT_ID	
- DIV4_AIRPORT_SEQ_ID	
- DIV4_WHEELS_ON	
- DIV4_TOTAL_GTIME	
- DIV4_LONGEST_GTIME	
- DIV4_WHEELS_OFF	
- DIV4_TAIL_NUM	
- DIV5_AIRPORT	
- DIV5_AIRPORT_ID	
- DIV5_AIRPORT_SEQ_ID	
- DIV5_WHEELS_ON	
- DIV5_TOTAL_GTIME	
- DIV5_LONGEST_GTIME	
- DIV5_WHEELS_OFF	
- DIV5_TAIL_NUM

DIV4 and DIV5 related variables do not have data, so we will remove them.

In [77]:
# check if there is values from DIV1 to DIV5 related variables
df_airlines.describe()

OP_CARRIER_AIRLINE_ID 
 OP_CARRIER_FL_NUM 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY_NEW 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY_NEW 
 CANCELLED 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DELAY_15 
 
 
 
 
 count 
 161057.000000 
 161057.000000 
 161057.000000 
 156231.000000 
 156231.000000 
 156121.000000 
 156121.000000 
 155957.000000 
 155957.000000 
 161057.000000 
 155957.000000 
 155692.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 155692.000000 
 155692.000000 
 161057.0 
 161057.000000 
 34915.000000 
 34915.000000 
 34915.00000 
 34915.000000 
 34915.000000 
 1053.000000 
 1053.000000 
 1053.000000 
 161057.000000 
 381.000000 
 265.000000 
 265.000000 
 381.000000 
 429.000000 
 4.290000e+02 
 429.000000 
 429.000000 
 429.000000 
 278.000000 
 17.000000 
 1.700000e+01 
 17.000000 
 17.000000 
 17.000000 
 5.000000 
 1.0 
 1.0 
 1.0 
 1.0 
 1.0 
 161057.000000 
 
 
 mean 
 20010.918153 
 2508.798711 
 1422.252768 
 1437.216897 
 14.368870 
 18.062086 
 1465.591503 
 1563.578172 
 6.149086 
 1584.805361 
 1566.326064 
 14.163483 
 0.030946 
 0.002366 
 124.434784 
 120.037144 
 95.836177 
 1.0 
 662.231340 
 19.903050 
 6.769039 
 12.16838 
 0.017557 
 19.076672 
 1399.992403 
 36.226021 
 35.970560 
 0.005116 
 0.695538 
 347.101887 
 232.633962 
 47.908136 
 12499.242424 
 1.249927e+06 
 1464.046620 
 27.405594 
 22.242424 
 1474.884892 
 12674.176471 
 1.267421e+06 
 1341.411765 
 11.941176 
 10.235294 
 1392.200000 
 10397.0 
 1039705.0 
 51.0 
 4.0 
 4.0 
 0.227143 
 
 
 std 
 314.626497 
 1746.419264 
 453.970474 
 463.857461 
 35.596949 
 9.087123 
 465.871793 
 488.380807 
 4.818137 
 467.098093 
 494.175085 
 36.196183 
 0.173171 
 0.048580 
 64.537094 
 63.288804 
 61.754708 
 0.0 
 474.807024 
 38.098095 
 29.870353 
 25.07071 
 0.805233 
 36.269235 
 482.825093 
 28.256333 
 27.885904 
 0.163356 
 0.460784 
 204.471624 
 206.620475 
 124.824801 
 1657.100266 
 1.657094e+05 
 564.358918 
 25.880631 
 24.029032 
 543.504354 
 1805.210349 
 1.805201e+05 
 617.786073 
 9.222734 
 7.198345 
 586.312374 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0.418987 
 
 
 min 
 19393.000000 
 3.000000 
 500.000000 
 1.000000 
 0.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 0.000000 
 0.000000 
 0.000000 
 37.000000 
 27.000000 
 14.000000 
 1.0 
 67.000000 
 0.000000 
 0.000000 
 0.00000 
 0.000000 
 0.000000 
 25.000000 
 1.000000 
 1.000000 
 0.000000 
 0.000000 
 91.000000 
 -6.000000 
 0.000000 
 10135.000000 
 1.013503e+06 
 6.000000 
 2.000000 
 2.000000 
 11.000000 
 10397.000000 
 1.039705e+06 
 50.000000 
 3.000000 
 3.000000 
 824.000000 
 10397.0 
 1039705.0 
 51.0 
 4.0 
 4.0 
 0.000000 
 
 
 25% 
 19790.000000 
 1171.000000 
 1025.000000 
 1029.000000 
 0.000000 
 13.000000 
 1046.000000 
 1155.000000 
 4.000000 
 1206.000000 
 1159.000000 
 0.000000 
 0.000000 
 0.000000 
 81.000000 
 78.000000 
 55.000000 
 1.0 
 332.000000 
 0.000000 
 0.000000 
 0.00000 
 0.000000 
 0.000000 
 953.000000 
 19.000000 
 19.000000 
 0.000000 
 0.000000 
 231.000000 
 127.000000 
 0.000000 
 10821.000000 
 1.082103e+06 
 1045.000000 
 10.000000 
 9.000000 
 1121.250000 
 10397.000000 
 1.039705e+06 
 1025.000000 
 7.000000 
 7.000000 
 1018.000000 
 10397.0 
 1039705.0 
 51.0 
 4.0 
 4.0 
 0.000000 
 
 
 50% 
 19805.000000 
 1990.000000 
 1425.000000 
 1444.00000

In [78]:
# delete diverted airport information related redundant variables
del_list_div_info = ['DIV4_AIRPORT', 'DIV4_AIRPORT_ID', 'DIV4_AIRPORT_SEQ_ID', 'DIV4_WHEELS_ON', 'DIV4_TOTAL_GTIME', 'DIV4_LONGEST_GTIME', 'DIV4_WHEELS_OFF', 'DIV4_TAIL_NUM', 'DIV5_AIRPORT', 'DIV5_AIRPORT_ID', 'DIV5_AIRPORT_SEQ_ID', 'DIV5_WHEELS_ON', 'DIV5_TOTAL_GTIME', 'DIV5_LONGEST_GTIME', 'DIV5_WHEELS_OFF', 'DIV5_TAIL_NUM']

for col in del_list_div_info:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY_NEW 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY_NEW 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 5.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 16.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 6.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 32.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 23.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0

#### 2-2. Check the highly correlated variables from correlation matrix

In [80]:
##Correlation Matrix
air_corr = df_airlines.corr()
fig, ax = plt.subplots(figsize=(20, 20))
mask = np.zeros_like(air_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sn.diverging_palette(240, 10, as_cmap=True)
sn.heatmap(air_corr, mask=mask, cmap=cmap, center=0, linewidths=.5)
plt.title("Correlations between features.")
display(plt.show())

#### 2-3. Remove the variables which is related to the outcome "DELAY_15"
- DEP_DELAY_NEW
- ARR_DELAY_NEW

In [82]:
# delete outcome related variables
del_list_outcome_related = ['DEP_DELAY_NEW', 'ARR_DELAY_NEW']

for col in del_list_outcome_related:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0

In [83]:
df_airlines.shape

Out[54]: (161057, 62)

### 3. Normalization of Data

##### Standardize continuous variables
- CRS_ELAPSED_TIME	
- ACTUAL_ELAPSED_TIME	
- AIR_TIME 
- DISTANCE	
- CARRIER_DELAY	
- WEATHER_DELAY	
- NAS_DELAY	
- SECURITY_DELAY	
- LATE_AIRCRAFT_DELAY 
- DIV_ACTUAL_ELAPSED_TIME 
- DIV_ARR_DELAY	
- DIV_DISTANCE

Some of the variables in airlines is date, str or categorical data, so we just select continuous variables to standardize.

In [86]:
# import Standardize package
from sklearn.preprocessing import StandardScaler

In [87]:
# Get continuous variables
X_continuous =df_airlines[['CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'DIV_ACTUAL_ELAPSED_TIME', 'DIV_ARR_DELAY', 'DIV_DISTANCE']]
X_continuous.head()

CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 
 
 
 
 0 
 160.0 
 165.0 
 144.0 
 1120.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 1 
 160.0 
 166.0 
 152.0 
 1120.0 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 
 
 2 
 160.0 
 196.0 
 154.0 
 1120.0 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 
 
 3 
 160.0 
 196.0 
 169.0 
 1120.0 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 
 
 4 
 160.0 
 171.0 
 145.0 
 1120.0 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN

In [88]:
# Standardizing the continuous variables
X = StandardScaler().fit_transform(X_continuous)
X

Out[57]: array([[ 0.55108348, 0.71044162, 0.77992398, ..., nan,
 nan, nan],
 [ 0.55108348, 0.72624226, 0.90946918, ..., nan,
 nan, nan],
 [ 0.55108348, 1.20026127, 0.94185548, ..., nan,
 nan, nan],
 ...,
 [-0.61104249, -0.58521035, -0.43456221, ..., nan,
 nan, nan],
 [-0.53356742, -0.68001415, -0.54791426, ..., nan,
 nan, nan],
 [-0.53356742, -0.60101098, -0.46694851, ..., nan,
 nan, nan]])

### 4. Dimensionality reduction

#### 4-1. Use PCA for continuous variables

In [91]:
# import PCA package
from sklearn.decomposition import PCA

In [92]:
# fill in all na value to 0 because PCA can't work on NaN value
X[np.isnan(X)] = 0

In [93]:
# Use PCA, reduce the feature vector to 2D
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
df_principal = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
df_principal.head()

principal component 1 
 principal component 2 
 
 
 
 
 0 
 1.502625 
 0.007499 
 
 
 1 
 1.575095 
 0.268880 
 
 
 2 
 1.835153 
 -0.554518 
 
 
 3 
 1.966342 
 -1.303872 
 
 
 4 
 1.559364 
 0.293698

#### 4-2. Combine the airlines together (PCA continuous variables + non-continuous variables)

In [95]:
# get non-continuous variables
df_airlines_non_continuous = df_airlines.drop(['CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'DIV_ACTUAL_ELAPSED_TIME', 'DIV_ARR_DELAY', 'DIV_DISTANCE'], axis=1)
df_airlines_non_continuous.shape

Out[61]: (161057, 50)

In [96]:
# concat airlines PCA continuous and non-continuous variables
df_airlines_pca = pd.concat([df_airlines_non_continuous, df_principal], axis=1)
df_airlines_pca.shape

Out[62]: (161057, 52)

### 5. Joining the Datasets
Join the airline and weather dataset by location and time
- Location:
- Time: 
The flight data needs to be joined to the weather data, but the weather data must be two hours ahead of the flight departure

#### 5-1. Check weather data's location and time
- Location: CALL_SIGN -> ORIGIN
- Time: DATE +2hr -> DEP_HOUR, FL_DATE

In [99]:
weather.select('CALL_SIGN').distinct().collect()

Out[110]: [Row(CALL_SIGN='KCEY '),
 Row(CALL_SIGN='KPRX '),
 Row(CALL_SIGN='KGGP '),
 Row(CALL_SIGN='KMGJ '),
 Row(CALL_SIGN='KBHB '),
 Row(CALL_SIGN='KTZR '),
 Row(CALL_SIGN='KLBE '),
 Row(CALL_SIGN='TJNR '),
 Row(CALL_SIGN='KRQB '),
 Row(CALL_SIGN='KMGR '),
 Row(CALL_SIGN='KBYI '),
 Row(CALL_SIGN='KFAR '),
 Row(CALL_SIGN='PAFM '),
 Row(CALL_SIGN='PALJ '),
 Row(CALL_SIGN='KGYB '),
 Row(CALL_SIGN='KGMU '),
 Row(CALL_SIGN='KLAN '),
 Row(CALL_SIGN='KPDT '),
 Row(CALL_SIGN='KMRN '),
 Row(CALL_SIGN='KBQK '),
 Row(CALL_SIGN='KAVK '),
 Row(CALL_SIGN='KPTW '),
 Row(CALL_SIGN='KDMN '),
 Row(CALL_SIGN='KBXA '),
 Row(CALL_SIGN='KLKU '),
 Row(CALL_SIGN='KCIR '),
 Row(CALL_SIGN='KGCC '),
 Row(CALL_SIGN='KMMU '),
 Row(CALL_SIGN='KHOE '),
 Row(CALL_SIGN='KSBA '),
 Row(CALL_SIGN='KPUJ '),
 Row(CALL_SIGN='KOAJ '),
 Row(CALL_SIGN='KHST '),
 Row(CALL_SIGN='PADE '),
 Row(CALL_SIGN='K40B '),
 Row(CALL_SIGN='KCKP '),
 Row(CALL_SIGN='PABT '),
 Row(CALL_SIGN='K14Y '),
 Row(CALL_SIGN='KDYL '),
 Row(CALL_SIGN='KSAZ '),
 Row(CALL_SIGN='KBEH '),
 Row(CALL_SIGN='KALM '),
 Row(CALL_SIGN='KPNE '),
 Row(CALL_SIGN='KROA '),
 Row(CALL_SIGN='KCNK '),
 Row(CALL_SIGN='KHYW '),
 Row(CALL_SIGN='KFYG '),
 Row(CALL_SIGN='KRSW '),
 Row(CALL_SIGN='KIKV '),
 Row(CALL_SIGN='KARA '),
 Row(CALL_SIGN='KTOI '),
 Row(CALL_SIGN='KMSO '),
 Row(CALL_SIGN='KEHO '),
 Row(CALL_SIGN='K8A0 '),
 Row(CALL_SIGN='KFKA '),
 Row(CALL_SIGN='KCMR '),
 Row(CALL_SIGN='KAIK '),
 Row(CALL_SIGN='KGRY '),
 Row(CALL_SIGN='KPNM '),
 Row(CALL_SIGN='KSSC '),
 Row(CALL_SIGN='KGIF '),
 Row(CALL_SIGN='KNFG '),
 Row(CALL_SIGN='KATY '),
 Row(CALL_SIGN='KHAE '),
 Row(CALL_SIGN='KGCD '),
 Row(CALL_SIGN='KELN '),
 Row(CALL_SIGN='KMHV '),
 Row(CALL_SIGN='PAMC '),
 Row(CALL_SIGN='KRGK '),
 Row(CALL_SIGN='KTRI '),
 Row(CALL_SIGN='KHDC '),
 Row(CALL_SIGN='KCOT '),
 Row(CALL_SIGN='KPRN '),
 Row(CALL_SIGN='KTRL '),
 Row(CALL_SIGN='KSIY '),
 Row(CALL_SIGN='K65S '),
 Row(CALL_SIGN='KSMX '),
 Row(CALL_SIGN='KNFW '),
 Row(CALL_SIGN='KWHP '),
 Row(CALL_SIGN='KZZV '),
 Row(CALL_SIGN='KDEH '),
 Row(CALL_SIGN='KROC '),
 Row(CALL_SIGN='PAOR '),
 Row(CALL_SIGN='KMPZ '),
 Row(CALL_SIGN='KM40 '),
 Row(CALL_SIGN='KSRR '),
 Row(CALL_SIGN='KDDH '),
 Row(CALL_SIGN='KPWM '),
 Row(CALL_SIGN='KSFM '),
 Row(CALL_SIGN='KMKL '),
 Row(CALL_SIGN='KCPW '),
 Row(CALL_SIGN='KVRB '),
 Row(CALL_SIGN='KRAS '),
 Row(CALL_SIGN='K1A6 '),
 Row(CALL_SIGN='KCKV '),
 Row(CALL_SIGN='KBUY '),
 Row(CALL_SIGN='KLWS '),
 Row(CALL_SIGN='KUGN '),
 Row(CALL_SIGN='K12N '),
 Row(CALL_SIGN='TJIG '),
 Row(CALL_SIGN='PAGL '),
 Row(CALL_SIGN='KUCY '),
 Row(CALL_SIGN='KD95 '),
 Row(CALL_SIGN='K2G4 '),
 Row(CALL_SIGN='KH78 '),
 Row(CALL_SIGN='KDPA '),
 Row(CALL_SIGN='KMLE '),
 Row(CALL_SIGN='KSWW '),
 Row(CALL_SIGN='KW22 '),
 Row(CALL_SIGN='KMLT '),
 Row(CALL_SIGN='PAAK '),
 Row(CALL_SIGN='KTTD '),
 Row(CALL_SIGN='KLBF '),
 Row(CALL_SIGN='KP58 '),
 Row(CALL_SIGN='KECP '),
 Row(CALL_SIGN='KWLD '),
 Row(CALL_SIGN='KMNI '),
 Row(CALL_SIGN='KBLV '),
 Row(CALL_SIGN='K05U '),
 Row(CALL_SIGN='KDLN '),
 Row(CALL_SIGN='K19A '),
 Row(CALL_SIGN='KMTJ '),
 Row(CALL_SIGN='KHRI '),
 Row(CALL_SIGN='KCRE '),
 Row(CALL_SIGN='KHLX '),
 Row(CALL_SIGN='KPAE '),
 Row(CALL_SIGN='KLAS '),
 Row(CALL_SIGN='KDBQ '),
 Row(CALL_SIGN='KBLF '),
 Row(CALL_SIGN='KAQP '),
 Row(CALL_SIGN='KEMT '),
 Row(CALL_SIGN='KCRS '),
 Row(CALL_SIGN='KDTN '),
 Row(CALL_SIGN='KASG '),
 Row(CALL_SIGN='KSQL '),
 Row(CALL_SIGN='KGDV '),
 Row(CALL_SIGN='KSUX '),
 Row(CALL_SIGN='KDFI '),
 Row(CALL_SIGN='KBKE '),
 Row(CALL_SIGN='PATO '),
 Row(CALL_SIGN='PASD '),
 Row(CALL_SIGN='PALH '),
 Row(CALL_SIGN='KDAL '),
 Row(CALL_SIGN='KW99 '),
 Row(CALL_SIGN='KRYT '),
 Row(CALL_SIGN='KMPO '),
 Row(CALL_SIGN='KCHD '),
 Row(CALL_SIGN='KDGW '),
 Row(CALL_SIGN='KPVW '),
 Row(CALL_SIGN='KALN '),
 Row(CALL_SIGN='KIDP '),
 Row(CALL_SIGN='KAMW '),
 Row(CALL_SIGN='KPQL '),
 Row(CALL_SIGN='KCKC '),
 Row(CALL_SIGN='KENW '),
 Row(CALL_SIGN='KSDL '),
 Row(CALL_SIGN='KGUY '),
 Row(CALL_SIGN='KGEY '),
 Row(CALL_SIGN='PGRO '),
 Row(CALL_SIGN=

In [100]:
# Select CALL_SIGN near ORD, ATL
# ATL closest station is KATL
# ORD closest station is KORD
weather_loc = weather.where(f.col("CALL_SIGN").isin({"KATL ", "KORD "}))
weather_loc.count()

Out[111]: 5863

In [101]:
# Change the weather_loc to dataframe
df_weather = weather_loc.toPandas()
df_weather.shape

Out[112]: (5863, 177)

In [102]:
df_weather.shape

Out[113]: (5863, 177)

In [103]:
df_weather.describe()

SOURCE 
 LATITUDE 
 LONGITUDE 
 ELEVATION 
 
 
 
 
 count 
 5863.000000 
 5863.000000 
 5863.000000 
 5863.000000 
 
 
 mean 
 6.936722 
 37.924548 
 -86.234452 
 253.380761 
 
 
 std 
 0.243483 
 4.181307 
 1.745423 
 52.985513 
 
 
 min 
 6.000000 
 33.630100 
 -87.933600 
 201.800000 
 
 
 25% 
 7.000000 
 33.630100 
 -87.933600 
 201.800000 
 
 
 50% 
 7.000000 
 41.995000 
 -87.933600 
 201.800000 
 
 
 75% 
 7.000000 
 41.995000 
 -84.441800 
 307.800000 
 
 
 max 
 7.000000 
 41.995000 
 -84.441800 
 307.800000

In [104]:
df_weather.head()

STATION 
 DATE 
 SOURCE 
 LATITUDE 
 LONGITUDE 
 ELEVATION 
 NAME 
 REPORT_TYPE 
 CALL_SIGN 
 QUALITY_CONTROL 
 WND 
 CIG 
 VIS 
 TMP 
 DEW 
 SLP 
 AW1 
 GA1 
 GA2 
 GA3 
 GA4 
 GE1 
 GF1 
 KA1 
 KA2 
 MA1 
 MD1 
 MW1 
 MW2 
 OC1 
 OD1 
 OD2 
 REM 
 EQD 
 AW2 
 AX4 
 GD1 
 AW5 
 GN1 
 AJ1 
 ... 
 GA6 
 KG1 
 AU3 
 AT7 
 KD1 
 GL1 
 IA1 
 GG2 
 OD3 
 UG1 
 CB1 
 AI6 
 CI1 
 CV2 
 AZ2 
 AD1 
 AH1 
 WD1 
 AA4 
 KC1 
 IA2 
 CF3 
 AI2 
 AT1 
 GD4 
 AX3 
 AH4 
 KB3 
 CU1 
 CN4 
 AT2 
 CG1 
 CF1 
 GG1 
 MV2 
 CW1 
 GG4 
 AB1 
 AH5 
 CN3 
 
 
 
 
 0 
 72530094846 
 2015-01-01 00:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0057,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10234,5 
 
 02,5,+07620,5,99,9 
 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,07620,1,99,9,99,9 
 
 
 10220,5,09974,5 
 
 
 
 0118,5 
 
 
 MET10412/31/14 18:51:03 METAR KORD 010051Z 240... 
 D01 0ADE726 
 
 
 1,99,1,+07620,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 72530094846 
 2015-01-01 01:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 230,5,N,0077,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0167,5 
 10235,5 
 
 02,5,+03048,5,99,9 
 02,5,+07620,5,99,9 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,03048,1,99,9,99,9 
 
 
 10220,5,09974,5 
 
 
 
 0113,5 
 4,99,0139,1,230 
 
 MET12912/31/14 19:51:03 METAR KORD 010151Z 230... 
 D01 0ADE726 
 
 
 1,99,1,+03048,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 72530094846 
 2015-01-01 02:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0072,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10232,5 
 
 02,5,+07620,5,99,9 
 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,07620,1,99,9,99,9 
 
 
 10220,5,09974,5 
 6,9,009,9,+999,9 
 
 
 0139,5 
 
 
 MET12812/31/14 20:51:03 METAR KORD 010251Z 240... 
 D01 0ADE726 
 
 
 1,99,1,+07620,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 72530094846 
 2015-01-01 03:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0067,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10229,5 
 
 00,5,+99999,9,99,9 
 
 
 
 
 00,99,1,99,9,99,9,99999,9,99,9,99,9 
 
 
 10217,5,09971,5 
 
 
 
 0103,5 
 4,99,0133,1,240 
 
 MET11912/31/14 21:51:03 METAR KORD 010351Z 240... 
 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4 
 72530094846 
 2015-01-01 04:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0088,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10223,5 
 
 00,5,+99999,9,99,9 
 
 
 
 
 00,99,1,99,9,99,9,99999,9,99,9,99,9 
 
 
 10210,5,09964,5 
 
 
 
 0124,5 
 
 
 MET10112/31/14 22:51:03 METAR KORD 010451Z 240... 
 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 rows × 177 columns

In [105]:
for col in df_weather.columns:
  print(col, df_weather[col].nunique())

STATION 2
DATE 5849
SOURCE 2
LATITUDE 2
LONGITUDE 2
ELEVATION 2
NAME 2
REPORT_TYPE 4
CALL_SIGN 2
QUALITY_CONTROL 1
WND 636
CIG 98
VIS 28
TMP 122
DEW 116
SLP 384
AW1 20
GA1 329
GA2 231
GA3 120
GA4 1
GE1 2
GF1 220
KA1 194
KA2 185
MA1 213
MD1 237
MW1 2
MW2 1
OC1 26
OD1 125
OD2 1
REM 5863
EQD 170
AW2 27
AX4 2
GD1 343
AW5 2
GN1 1
AJ1 37
AW3 11
MK1 7
KA4 40
GG3 1
AN1 30
RH1 45
AU5 1
HL1 1
OB1 1
AT8 1
AW7 1
AZ1 1
CH1 1
RH3 58
GK1 1
IB1 1
AX1 5
CT1 1
AK1 4
CN2 1
OE1 165
MW5 1
AO1 1
KA3 45
AA3 31
CR1 1
CF2 1
KB2 7
GM1 1
AT5 5
AY2 1
MW6 1
MG1 178
AH6 6
AU2 17
GD2 221
AW4 4
MF1 164
AA1 103
AH2 6
AH3 6
OE3 106
AT6 5
AL2 1
AL3 1
AX5 1
IB2 1
AI3 6
CV3 1
WA1 5
GH1 1
KF1 1
CU2 1
CT3 1
SA1 1
AU1 32
KD2 3
AI5 6
GO1 1
GD3 120
CG3 1
AI1 6
AL1 1
AW6 1
MW4 1
AX6 1
CV1 1
ME1 1
KC2 7
CN1 1
UA1 1
GD5 1
UG2 1
AT3 13
AT4 7
GJ1 1
MV1 1
GA5 1
CT2 1
CG2 1
ED1 3
AE1 7
CO1 1
KE1 7
KB1 7
AI4 6
MW3 1
KG2 63
AA2 82
AX2 10
AY1 1
RH2 68
OE2 146
CU3 1
MH1 7
AM1 6
AU4 2
GA6 1
KG1 59
AU3 6
AT7 4
KD1 7
GL1 1
IA1 1
GG2 1
OD3 1
UG1 1
CB1 1
AI6 6
CI1 1
CV2 1
AZ2 1
AD1 7
AH1 6
WD1 1
AA4 1
KC1 7
IA2 1
CF3 1
AI2 6
AT1 9
GD4 19
AX3 5
AH4 6
KB3 7
CU1 1
CN4 1
AT2 13
CG1 1
CF1 1
GG1 1
MV2 1
CW1 1
GG4 1
AB1 7
AH5 6
CN3 1

In [106]:
df_weather.MW1.value_counts()

Out[117]: 5842
05,5 21
Name: MW1, dtype: int64

In [107]:
cols_to_delete = ['GA4','GE1','MW2','OD2','GN1','GG3','AU5','HL1','OB1','AT8','AW7','AZ1','CH1','GK1','IB1','CT1','CN2','MW5','AO1','CR1','CF2','GM1','AY2','MW6',
                 'AL2','AL3','AX5','IB2','CV3','GH1','KF1','CU2','CT3','SA1','GO1','CG3','AL1','AW6','MW4','AX6','CV1','ME1','CN1','UA1','GD5','UG2','GJ1','MV1',
                 'GA5','CT2','CG2','CO1','MW3','AY1','CU3','GA6','GL1','IA1','GG2','OD3','UG1','CB1','CI1','CV2','AZ2','WD1','AA4','IA2','CF3','CU1','CN4','CG1',
                 'CF1','GG1','MV2','CW1','GG4','CN3']


In [108]:
for col in cols_to_delete:
  del df_weather[col]

for col in df_weather.columns:
  print(col, df_weather[col].nunique())

STATION 2
DATE 5849
SOURCE 2
LATITUDE 2
LONGITUDE 2
ELEVATION 2
NAME 2
REPORT_TYPE 4
CALL_SIGN 2
QUALITY_CONTROL 1
WND 636
CIG 98
VIS 28
TMP 122
DEW 116
SLP 384
AW1 20
GA1 329
GA2 231
GA3 120
GF1 220
KA1 194
KA2 185
MA1 213
MD1 237
MW1 2
OC1 26
OD1 125
REM 5863
EQD 170
AW2 27
AX4 2
GD1 343
AW5 2
AJ1 37
AW3 11
MK1 7
KA4 40
AN1 30
RH1 45
RH3 58
AX1 5
AK1 4
OE1 165
KA3 45
AA3 31
KB2 7
AT5 5
MG1 178
AH6 6
AU2 17
GD2 221
AW4 4
MF1 164
AA1 103
AH2 6
AH3 6
OE3 106
AT6 5
AI3 6
WA1 5
AU1 32
KD2 3
AI5 6
GD3 120
AI1 6
KC2 7
AT3 13
AT4 7
ED1 3
AE1 7
KE1 7
KB1 7
AI4 6
KG2 63
AA2 82
AX2 10
RH2 68
OE2 146
MH1 7
AM1 6
AU4 2
KG1 59
AU3 6
AT7 4
KD1 7
AI6 6
AD1 7
AH1 6
KC1 7
AI2 6
AT1 9
GD4 19
AX3 5
AH4 6
KB3 7
AT2 13
AB1 7
AH5 6

In [109]:
df_weather.AA3.value_counts()

Out[124]: 5822
24,0025,9,1 3
24,0018,9,1 3
24,0008,9,1 3
24,0015,9,1 2
24,0023,9,1 2
24,0003,9,1 2
24,0074,9,1 2
24,0048,9,1 2
24,0318,9,1 1
24,0193,9,1 1
24,0051,9,1 1
24,0135,9,1 1
24,0010,9,1 1
24,0041,9,1 1
24,0046,9,1 1
24,0089,9,1 1
24,0401,9,1 1
24,0000,2,1 1
24,0241,9,1 1
24,0066,9,1 1
24,0028,9,1 1
24,0086,9,1 1
24,0165,9,1 1
24,0036,9,1 1
24,0145,9,1 1
24,0069,9,1 1
24,0249,9,1 1
24,0079,9,1 1
24,0236,9,1 1
24,0175,9,1 1
Name: AA3, dtype: int64

In [110]:
cols_to_condense = ['AW1','AW3','AW4','AW5','GA1','GA2','GA3','GF1','MW1','MA1','MK1','MG1','MH1','OC1','GD1','GD2','GD4','GD3','AX1','AX2','AX3','AX4','AJ1','AK1',
                    'AI1','AI2','AI3','AI4','AI5','AI6','MF1','AE1','AM1','AD1','AB1']
cols_to_condense1 = ['KA1','KA2','KA3','KA4','MD1','OD1','RH1','RH2','RH3','OE1','OE2','OE3','KB1','KB2','KB3','AT7','AT1','AT2','AT3','AT4','AT5','AT6','KC1','KC2',
                     'ED1','KG1','KG2',] #for KA do index 2 
cols_to_condense2 = ['AA1','AA2','AA3','AH1','AH2','AH3','AH4','AH5','AH6','WA1','KD1','KD2',] #index 1


def condense_col(df, col_name,num):
  df[col_name] = df[col_name].str.split(',').str[num]
  return df

df_weather = condense_col(df_weather, cols_to_condense,0)
df_weather = condense_col(df_weather, cols_to_condense1,2)
df_weather = condense_col(df_weather, cols_to_condense2,1)
df_weather.head()

In [111]:
# Add ORIGIN column in weather to represent the location
df_weather.loc[df_weather['CALL_SIGN'] == 'KATL ', 'ORIGIN'] = 'ATL' 
df_weather.loc[df_weather['CALL_SIGN'] == 'KORD ', 'ORIGIN'] = 'ORD' 

In [112]:
# Add 2 hours in weather data because we want the predict the delay 2 hours before departure. Add DEP_HOUR and DEP_DATE columns in weather to represent the departure time
df_weather['DEP_HOUR'] = df_weather.DATE.map(lambda x: (x + datetime.timedelta(hours=2)).hour)
df_weather['FL_DATE'] = df_weather.DATE.map(lambda x: (x + datetime.timedelta(hours=2)).date())
df_weather.head()

STATION 
 DATE 
 SOURCE 
 LATITUDE 
 LONGITUDE 
 ELEVATION 
 NAME 
 REPORT_TYPE 
 CALL_SIGN 
 QUALITY_CONTROL 
 WND 
 CIG 
 VIS 
 TMP 
 DEW 
 SLP 
 AW1 
 GA1 
 GA2 
 GA3 
 GA4 
 GE1 
 GF1 
 KA1 
 KA2 
 MA1 
 MD1 
 MW1 
 MW2 
 OC1 
 OD1 
 OD2 
 REM 
 EQD 
 AW2 
 AX4 
 GD1 
 AW5 
 GN1 
 AJ1 
 ... 
 AT7 
 KD1 
 GL1 
 IA1 
 GG2 
 OD3 
 UG1 
 CB1 
 AI6 
 CI1 
 CV2 
 AZ2 
 AD1 
 AH1 
 WD1 
 AA4 
 KC1 
 IA2 
 CF3 
 AI2 
 AT1 
 GD4 
 AX3 
 AH4 
 KB3 
 CU1 
 CN4 
 AT2 
 CG1 
 CF1 
 GG1 
 MV2 
 CW1 
 GG4 
 AB1 
 AH5 
 CN3 
 DEP_HOUR 
 FL_DATE 
 ORIGIN 
 
 
 
 
 0 
 72530094846 
 2015-01-01 00:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0057,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10234,5 
 
 02,5,+07620,5,99,9 
 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,07620,1,99,9,99,9 
 
 
 10220,5,09974,5 
 
 
 
 0118,5 
 
 
 MET10412/31/14 18:51:03 METAR KORD 010051Z 240... 
 D01 0ADE726 
 
 
 1,99,1,+07620,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 2015-01-01 
 ORD 
 
 
 1 
 72530094846 
 2015-01-01 01:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 230,5,N,0077,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0167,5 
 10235,5 
 
 02,5,+03048,5,99,9 
 02,5,+07620,5,99,9 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,03048,1,99,9,99,9 
 
 
 10220,5,09974,5 
 
 
 
 0113,5 
 4,99,0139,1,230 
 
 MET12912/31/14 19:51:03 METAR KORD 010151Z 230... 
 D01 0ADE726 
 
 
 1,99,1,+03048,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 2015-01-01 
 ORD 
 
 
 2 
 72530094846 
 2015-01-01 02:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0072,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10232,5 
 
 02,5,+07620,5,99,9 
 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,07620,1,99,9,99,9 
 
 
 10220,5,09974,5 
 6,9,009,9,+999,9 
 
 
 0139,5 
 
 
 MET12812/31/14 20:51:03 METAR KORD 010251Z 240... 
 D01 0ADE726 
 
 
 1,99,1,+07620,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4 
 2015-01-01 
 ORD 
 
 
 3 
 72530094846 
 2015-01-01 03:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0067,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10229,5 
 
 00,5,+99999,9,99,9 
 
 
 
 
 00,99,1,99,9,99,9,99999,9,99,9,99,9 
 
 
 10217,5,09971,5 
 
 
 
 0103,5 
 4,99,0133,1,240 
 
 MET11912/31/14 21:51:03 METAR KORD 010351Z 240... 
 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 2015-01-01 
 ORD 
 
 
 4 
 72530094846 
 2015-01-01 04:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0088,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10223,5 
 
 00,5,+99999,9,99,9 
 
 
 
 
 00,99,1,99,9,99,9,99999,9,99,9,99,9 
 
 
 10210,5,09964,5 
 
 
 
 0124,5 
 
 
 MET10112/31/14 22:51:03 METAR KORD 010451Z 240... 
 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 6 
 2015-01-01 
 ORD 
 
 
 
 5 rows × 180 columns

#### 5-2. Check airline data's location and time
- Location: ORIGIN
- Time: 
  - CRS_DEP_TIME -> DEP_HOUR
  - FL_DATE

In [114]:
# Extract the departure hour and date from CRS_DEP_TIME column
df_airlines_pca['DEP_HOUR'] = df_airlines_pca.CRS_DEP_TIME.map(lambda x: pd.to_datetime(x, format='%H%M').hour)
df_airlines_pca.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 FLIGHTS 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 principal component 1 
 principal component 2 
 DEP_HOUR 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1.502625 
 0.007499 
 10 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1.575095 
 0.268880 
 10 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1.835153 
 -0.554518 
 10 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1.966342 
 -1.303872 
 10 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1.559364 
 0.293698 
 10

#### 5-3. Join the airline and weather data by location and time
- Location:
  - ORIGIN
- Time:
  - FL_DATE
  - DEP_HOUR

##### Check FL_DATE

In [117]:
print(df_airlines_pca.FL_DATE[0], type(df_airlines_pca.FL_DATE[0]))
df_airlines_pca.FL_DATE.value_counts()

2015-01-01 <class 'str'>
Out[78]: 2015-03-30 1988
2015-03-13 1979
2015-03-27 1977
2015-03-20 1975
2015-03-16 1975
 ... 
2015-01-24 1337
2015-01-10 1334
2015-02-07 1331
2015-01-17 1287
2015-01-31 1270
Name: FL_DATE, Length: 90, dtype: int64

In [118]:
print(df_weather.FL_DATE[0], type(df_weather.FL_DATE[0]))
df_weather.FL_DATE.value_counts()

2015-01-01 <class 'datetime.date'>
Out[79]: 2015-02-26 132
2015-01-04 124
2015-03-03 106
2015-03-23 101
2015-01-12 100
 ... 
2015-02-06 50
2015-01-07 50
2015-02-28 48
2015-01-01 48
2015-04-01 4
Name: FL_DATE, Length: 91, dtype: int64

##### Check ORIGIN

In [120]:
print(df_airlines_pca.ORIGIN[1], type(df_airlines_pca.ORIGIN[1]))
df_airlines_pca.ORIGIN.value_counts()

ORD <class 'str'>
Out[80]: ATL 89632
ORD 71425
Name: ORIGIN, dtype: int64

In [121]:
print(df_weather.ORIGIN[1], type(df_weather.ORIGIN[1]))
df_weather.ORIGIN.value_counts()

ORD <class 'str'>
Out[81]: ORD 3010
ATL 2853
Name: ORIGIN, dtype: int64

##### Check DEP_HOUR

In [123]:
print(df_airlines_pca.DEP_HOUR[1], type(df_airlines_pca.DEP_HOUR[1]))
df_airlines_pca.DEP_HOUR.value_counts()

10 <class 'numpy.int64'>
Out[82]: 13 12685
8 12447
9 12288
19 11836
10 11471
15 11342
16 11286
20 11254
12 9859
17 9647
18 8612
21 8220
14 7718
11 7492
7 6834
22 3584
6 3453
5 885
23 144
Name: DEP_HOUR, dtype: int64

In [124]:
print(df_weather.DEP_HOUR[1], type(df_weather.DEP_HOUR[1]))
df_weather.DEP_HOUR.value_counts()

3 <class 'numpy.int64'>
Out[83]: 6 319
7 318
16 263
13 258
17 257
12 250
22 246
21 245
11 245
14 244
15 240
18 239
20 239
8 238
10 236
1 235
19 234
9 232
23 231
0 228
4 220
2 219
5 218
3 209
Name: DEP_HOUR, dtype: int64

##### Merge airline and weather data

In [126]:
# left join of airline and weather data by location and time
df = pd.merge(df_airlines_pca, df_weather, how='left', on=['ORIGIN', 'FL_DATE', 'DEP_HOUR'])
print(df.shape)
df.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 FLIGHTS 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 ... 
 GA6 
 KG1 
 AU3 
 AT7 
 KD1 
 GL1 
 IA1 
 GG2 
 OD3 
 UG1 
 CB1 
 AI6 
 CI1 
 CV2 
 AZ2 
 AD1 
 AH1 
 WD1 
 AA4 
 KC1 
 IA2 
 CF3 
 AI2 
 AT1 
 GD4 
 AX3 
 AH4 
 KB3 
 CU1 
 CN4 
 AT2 
 CG1 
 CF1 
 GG1 
 MV2 
 CW1 
 GG4 
 AB1 
 AH5 
 CN3 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 
 5 rows × 230 columns

In [127]:
#pull out div variables
for col in df.columns:
  print(col, df[col].nunique())
#airlines_eda.ORIGIN.value_counts()

FL_DATE 90
OP_UNIQUE_CARRIER 13
OP_CARRIER_AIRLINE_ID 13
OP_CARRIER 13
TAIL_NUM 3907
OP_CARRIER_FL_NUM 3853
ORIGIN 2
DEST 192
CRS_DEP_TIME 978
DEP_TIME 1244
TAXI_OUT 137
WHEELS_OFF 1259
WHEELS_ON 1279
TAXI_IN 111
CRS_ARR_TIME 1055
ARR_TIME 1281
CANCELLED 2
CANCELLATION_CODE 3
DIVERTED 2
FLIGHTS 1
FIRST_DEP_TIME 650
TOTAL_ADD_GTIME 129
LONGEST_ADD_GTIME 126
DIV_AIRPORT_LANDINGS 4
DIV_REACHED_DEST 2
DIV1_AIRPORT 98
DIV1_AIRPORT_ID 98
DIV1_AIRPORT_SEQ_ID 98
DIV1_WHEELS_ON 348
DIV1_TOTAL_GTIME 93
DIV1_LONGEST_GTIME 80
DIV1_WHEELS_OFF 243
DIV1_TAIL_NUM 250
DIV2_AIRPORT 7
DIV2_AIRPORT_ID 7
DIV2_AIRPORT_SEQ_ID 7
DIV2_WHEELS_ON 17
DIV2_TOTAL_GTIME 10
DIV2_LONGEST_GTIME 10
DIV2_WHEELS_OFF 5
DIV2_TAIL_NUM 5
DIV3_AIRPORT 1
DIV3_AIRPORT_ID 1
DIV3_AIRPORT_SEQ_ID 1
DIV3_WHEELS_ON 1
DIV3_TOTAL_GTIME 1
DIV3_LONGEST_GTIME 1
DIV3_WHEELS_OFF 0
DIV3_TAIL_NUM 0
DELAY_15 2
principal component 1 140631
principal component 2 140631
DEP_HOUR 19
STATION 0
DATE 0
SOURCE 0
LATITUDE 0
LONGITUDE 0
ELEVATION 0
NAME 0
REPORT_TYPE 0
CALL_SIGN 0
QUALITY_CONTROL 0
WND 0
CIG 0
VIS 0
TMP 0
DEW 0
SLP 0
AW1 0
GA1 0
GA2 0
GA3 0
GA4 0
GE1 0
GF1 0
KA1 0
KA2 0
MA1 0
MD1 0
MW1 0
MW2 0
OC1 0
OD1 0
OD2 0
REM 0
EQD 0
AW2 0
AX4 0
GD1 0
AW5 0
GN1 0
AJ1 0
AW3 0
MK1 0
KA4 0
GG3 0
AN1 0
RH1 0
AU5 0
HL1 0
OB1 0
AT8 0
AW7 0
AZ1 0
CH1 0
RH3 0
GK1 0
IB1 0
AX1 0
CT1 0
AK1 0
CN2 0
OE1 0
MW5 0
AO1 0
KA3 0
AA3 0
CR1 0
CF2 0
KB2 0
GM1 0
AT5 0
AY2 0
MW6 0
MG1 0
AH6 0
AU2 0
GD2 0
AW4 0
MF1 0
AA1 0
AH2 0
AH3 0
OE3 0
AT6 0
AL2 0
AL3 0
AX5 0
IB2 0
AI3 0
CV3 0
WA1 0
GH1 0
KF1 0
CU2 0
CT3 0
SA1 0
AU1 0
KD2 0
AI5 0
GO1 0
GD3 0
CG3 0
AI1 0
AL1 0
AW6 0
MW4 0
AX6 0
CV1 0
ME1 0
KC2 0
CN1 0
UA1 0
GD5 0
UG2 0
AT3 0
AT4 0
GJ1 0
MV1 0
GA5 0
CT2 0
CG2 0
ED1 0
AE1 0
CO1 0
KE1 0
KB1 0
AI4 0
MW3 0
KG2 0
AA2 0
AX2 0
AY1 0
RH2 0
OE2 0
CU3 0
MH1 0
AM1 0
AU4 0
GA6 0
KG1 0
AU3 0
AT7 0
KD1 0
GL1 0
IA1 0
GG2 0
OD3 0
UG1 0
CB1 0
AI6 0
CI1 0
CV2 0
AZ2 0
AD1 0
AH1 0
WD1 0
AA4 0
KC1 0
IA2 0
CF3 0
AI2 0
AT1 0
GD4 0
AX3 0
AH4 0
KB3 0
CU1 0
CN4 0
AT2 0
CG1 0
CF1 0
GG1 0
MV2 0
CW1 0
GG4 0
AB1 0
AH5 0
CN3 0

####Phase II summary: Scalability, Efficiency, Distributed/parallel Training and Scoring Pipeline
1. Create new outcome variables "DELAY_15" to check if the departure time is delay over 15 mins.
2. Remove redundant variables and variables related to the outcome.
3. Standardize the continuous variables.
4. Dimensionality reduction use PCA.
5. Join the airlines and weather data by location and time.

# Algorithm Exploration
Apply 2 to 3 algorithms to the training set, and discuss expectations, trade-offs, and results. Do not over fine tune these. You will want to use this process to select a final algorithm which you will spend your efforts on fine tuning.

### Splitting the Data

We're splitting the data into 3 sets. A training set that will contain 60% of the data. A test set that will contain 20% of the data and be used to test the performance of our algorithms. Lastly, a hold out set of 10% of the data to be used as a final check of the algorithm once we have one that is performant.

In [131]:
## Training, Test, Holdout Splits
flight_trainingRDD, flight_testRDD, flight_holdoutRDD = airlines.randomSplit([0.6, 0.2, 0.2], seed = 42)

# Algorithm Implementation 
Create your own toy example that matches the dataset provided and use this toy example to explain the math behind the algorithm that you will perform. Apply your algorithm to the training dataset and evaluate your results on the test set.

# Conclusions
Report results and learnings for both the ML as well as the scalability.

# Application of Course Concepts 
Pick 3-5 key course concepts and discuss how your work on this assignment illustrates an understanding of these concepts.